In [19]:
## TODO CRIAR BANCO PARA INSERIR DADOS DAS SEVS DO TEIA E ALIMENTAR COM AS TRATATIVAS DOS RESULTADOS DO GAIA.
# from sqlalchemy import create_engine
# import pandas as pd

# engine = create_engine('sqlite:///my.db', echo=False)
# df = pd.DataFrame([[1,2],[1,2]], columns=['a', 'b'])

# df.to_sql('other_table', con=engine, if_exists='append',index=False)


In [39]:
import pandas as pd
from unidecode import unidecode
import numpy as np
import time
import pyodbc
import warnings
from viabilipy.NFV import extracao_teia, tratamento_gaia

from selenium import webdriver
from selenium.webdriver.chrome.service import Service 
from selenium.webdriver.support.ui import Select, WebDriverWait
import chromedriver_autoinstaller
service = Service(chromedriver_autoinstaller.install())

pd.set_option('display.max_columns', None)
pd.set_option('future.no_silent_downcasting', True)
pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')

try:
    conn_str = (r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)}; DBQ=C:\Users\F257064\Documents\MODELO NFV\NFV LIMPO\Base_Terceiros.accdb;PWD=maderomcbk')
    conn = pyodbc.connect(conn_str)
    provedor_ethernet = pd.read_sql('SELECT * FROM provedorxlocalidade_ethernet', conn)
    valores_ethernet = pd.read_sql('SELECT * FROM valores_ethernet', conn)
    status = pd.read_sql('SELECT * FROM status', conn)
    conn.close()
    status = status.fillna('OK')
except:
    provedor_ethernet = pd.read_excel('./arquivos/provedorxlocalidade_ethernet.xlsx')
    valores_ethernet = pd.read_excel('./arquivos/valores_ethernet.xlsx')
    status = pd.read_excel('./arquivos/status.xlsx')




In [40]:
municipio_estacao = pd.read_excel('./arquivos/bb_municipio_estacao.xlsx')

In [41]:


# provedor_ethernet.to_excel('./arquivos/provedorxlocalidade_ethernet.xlsx',index=False)
# valores_ethernet.to_excel('./arquivos/valores_ethernet.xlsx',index=False)
# status.to_excel('./arquivos/status.xlsx',index=False)

caminho_pasta_arquivos = r'C:\Users\F257064\Documents\Codes\NEW_NFV\retorno_gaia'
nome_arquivo_teia = r"C:\Users\F257064\Documents\Codes\atualizacao nfv automatico\NEW_NFV\arquivo_teste\ext_20250402_091602-197d95133d25c9a0847450c3d9a1eca4.csv"

capacity = pd.read_excel('./arquivos/capacity.xlsx')
municipio_estacao = pd.read_excel('./arquivos/bb_municipio_estacao.xlsx')
estacoes_uf = pd.read_excel('./arquivos/estacoes_entregas.xlsx')
cidades = pd.read_excel('./arquivos/cidades_bbip.xlsx')
custos_acesso = pd.read_excel('./arquivos/custos_nfv.xlsx')
tecnologia_facilidade = pd.read_excel('./arquivos/id_tecnologia_facilidades.xlsx')
id_provedores = pd.read_excel('./arquivos/id_provedores.xlsx')
capacity_fixa = pd.read_excel('./arquivos/capacity_fixa.xlsx')
tecnologia_capacity = pd.read_excel('./arquivos/tecnologia_capacity.xlsx')
capacity_funil = pd.read_excel('./arquivos/capacity_funil.xlsb')
estacoes_newteia = pd.read_excel('./arquivos/lista_estacoes_newteia.xlsx')

# teia = pd.read_csv(f'{caminho_pasta_arquivos}\{nome_arquivo_teia}',sep=';')
teia = pd.read_csv(nome_arquivo_teia,sep=';')
fechamento_teia = pd.DataFrame(columns=['sequencial','latitude','longitude','uf','cnl','facilidade','id_facilidade','provedor','id_provedor','entrega',
                                        'abordado','custo_de_acesso_proprio','instalacao_terceiros','mensalidade_terceiros','tipo_terceiros','id_da_sev','prazo',
                                        'bb_ip','hp_bsod','codigo_spe','sinalizacao_sip','protocolo_gaia','obs','justificativa','ID Justificativa','status'])

# teia = pd.read_excel(f'{caminho_pasta_arquivos}\extracao_teia.xlsx')

tratamento = extracao_teia.extracaoTeia(teia)
removidas =  tratamento.tratar_modelo_gaia(removed_sevs='S')
facilidades = pd.read_excel('./arquivos/facilidades_tecnologia_prioridade.xlsx').sort_values('PRIORIDADE',ascending=True).fillna('')
# facilidades = pd.read_excel('./arquivos/facilidades.xlsx').sort_values('PRIORIDADE',ascending=True).fillna('')




In [42]:
custos_acesso = pd.read_excel('./arquivos/custos_nfv.xlsx')

In [43]:

resumosoe = tratamento_gaia.tratamentoResumosoe(rf"C:\Users\F257064\Documents\Codes\atualizacao nfv automatico\NEW_NFV\arquivo_teste\resumoSoE 17.txt").trata_resumosoe()
resultado = tratamento_gaia.tratamentoResultado(rf"C:\Users\F257064\Documents\Codes\atualizacao nfv automatico\NEW_NFV\arquivo_teste\resultado 15.txt").trata_resultado()
restricao = tratamento_gaia.tratamentoRestricao(rf"C:\Users\F257064\Documents\Codes\atualizacao nfv automatico\NEW_NFV\arquivo_teste\restricao 14.txt").trata_restricao()
nuvens = tratamento_gaia.tratamentoNuvens(rf"C:\Users\F257064\Documents\Codes\atualizacao nfv automatico\NEW_NFV\arquivo_teste\nuvens 16.txt").trata_nuvens()
# nuvens_terceiros = tratamento_gaia.tratamentoNuvensTerceiros(rf"{caminho_pasta_arquivos}\nuvensTerceiros.txt").trata_nuvens()
sevs_tratar = teia.drop(index=teia[teia.SEV.isin(removidas)].index).reset_index(drop=True)

sevs_tratar['PONTA_A'] = sevs_tratar['PONTA_A'].fillna('')


In [50]:

acum_nuvens = pd.DataFrame(columns=nuvens.columns)

for i, v in nuvens.iterrows():
    try:
        tec = v.TECNOLOGIA.split(' / ')
        
        for value in tec:
            aux = nuvens.loc[i].to_dict()
            aux.update(TECNOLOGIA=value)
            acum_nuvens.loc[len(acum_nuvens)] = aux

    except:
        pass

nuvens = acum_nuvens.drop_duplicates()


for i, v in nuvens.iterrows():
    aux_capcity = tecnologia_capacity[tecnologia_capacity.TECNOLOGIA == v.TECNOLOGIA]
    if len(aux_capcity) > 0:
        if aux_capcity.CAPACITY.values[0] not in ['SIGLA_ESTACAO_CLARO','ESTACAO_ENTREGA']:
            if v.TECNOLOGIA == 'VIRTUA':
                if sevs_tratar[sevs_tratar.SEV == v.SEV].SERVICO.values[0] == 'VPE - VIP BSOD LIGHT':
                    nuvens.at[i,'CAPACITY_NUVEM'] = 1000
                else:
                    nuvens.at[i,'CAPACITY_NUVEM'] = 0
            else:
                nuvens.at[i,'CAPACITY_NUVEM'] = int(aux_capcity.CAPACITY.values[0])
        else:
            if aux_capcity.CAPACITY.values[0] == 'ESTACAO_ENTREGA':
            # CONSULTAR UTILIZANDO AS COLUNAS DA TABELA TECNOLOGIA_CAPACITY, DO FUNIL E TABELA DE CAPACITYS.
                if v.TECNOLOGIA == 'FO EDD NET':
                    if len(capacity[(capacity.NUVEM == 'FO EDD NET') & (capacity.ESTACAO_ENTREGA == v.ESTACAO_ENTREGA)]) > 0:
                        nuvens.at[i,'CAPACITY_NUVEM'] = capacity[(capacity.NUVEM == 'FO EDD NET') & (capacity.ESTACAO_ENTREGA == v.ESTACAO_ENTREGA)].CAPACITY_MB.values[0]
                        nuvens.at[i,'CENTRO_ROTEAMENTO'] = capacity[(capacity.NUVEM == 'FO EDD NET') & (capacity.ESTACAO_ENTREGA == v.ESTACAO_ENTREGA)].CENTRO_ROTEAMENTO.values[0]
                elif v.TECNOLOGIA == 'GPON NET':
                    if len(capacity[(capacity.NUVEM == 'FO EDD NET') & (capacity.ESTACAO_ENTREGA == v.ESTACAO_ENTREGA)]) > 0:
                        if capacity[(capacity.NUVEM == 'FO EDD NET') & (capacity.ESTACAO_ENTREGA == v.ESTACAO_ENTREGA)].CAPACITY_MB.values[0] > 200:
                            nuvens.at[i,'CAPACITY_NUVEM'] = 200
                            nuvens.at[i,'CENTRO_ROTEAMENTO'] = capacity[(capacity.NUVEM == 'FO EDD NET') & (capacity.ESTACAO_ENTREGA == v.ESTACAO_ENTREGA)].CENTRO_ROTEAMENTO.values[0]
                        else:
                            nuvens.at[i,'CAPACITY_NUVEM'] = capacity[(capacity.NUVEM == 'FO EDD NET') & (capacity.ESTACAO_ENTREGA == v.ESTACAO_ENTREGA)].CAPACITY_MB.values[0]
                elif v.TECNOLOGIA == 'SDH':
                    if len(capacity[(capacity.NUVEM == 'FO SDH') & (capacity.ESTACAO_ENTREGA == v.ESTACAO_ENTREGA)]) > 0:
                        nuvens.at[i,'CAPACITY_NUVEM'] = capacity[(capacity.NUVEM == 'FO SDH') & (capacity.ESTACAO_ENTREGA == v.ESTACAO_ENTREGA)].CAPACITY_MB.values[0]
                        nuvens.at[i,'CENTRO_ROTEAMENTO'] = capacity[(capacity.NUVEM == 'FO SDH') & (capacity.ESTACAO_ENTREGA == v.ESTACAO_ENTREGA)].CENTRO_ROTEAMENTO.values[0]
                else:
                    if len(capacity_fixa[(capacity_fixa.TECNOLOGIA == v.TECNOLOGIA) & (capacity_fixa.SIGLA_EMBRATEL == v.ESTACAO_ENTREGA)]) > 0:
                        nuvens.at[i,'CAPACITY_NUVEM'] = capacity_fixa[(capacity_fixa.TECNOLOGIA == v.TECNOLOGIA) & (capacity_fixa.SIGLA_EMBRATEL == v.ESTACAO_ENTREGA)].TOTAL.values[0]
                        nuvens.at[i,'CENTRO_ROTEAMENTO'] = capacity_fixa[(capacity_fixa.TECNOLOGIA == v.TECNOLOGIA) & (capacity_fixa.SIGLA_EMBRATEL == v.ESTACAO_ENTREGA)].ESTACAO_BB.values[0]
            elif aux_capcity.CAPACITY.values[0] == 'SIGLA_ESTACAO_CLARO':
                aux_funil = capacity_funil[capacity_funil.SITE == v.SIGLA_ESTACAO_CLARO]
                if len(aux_funil) > 0:
                    nuvens.at[i,'CENTRO_ROTEAMENTO'] = v.ESTACAO_ENTREGA
                    if nuvens.at[i,'REDE'] == 'CORTE CAPACIDADE-BANDA':
                        nuvens.at[i,'CAPACITY_NUVEM'] = 10
                    elif 'CORTE PLANEJAMENTO REGIONAL' in nuvens.at[i,'REDE']:
                        nuvens.at[i,'CAPACITY_NUVEM'] = 0
                    elif nuvens.at[i,'SITUACAO'] == 'ESGOTADA':
                        nuvens.at[i,'CAPACITY_NUVEM'] = 0
                    elif nuvens.at[i,'SITUACAO'] == 'CONCLUIDA':
                        if nuvens.at[i,'MEIO_TRANSMISSAO'] == 'REDE OPTICA':
                            nuvens.at[i,'CAPACITY_NUVEM'] = 100
                        elif nuvens.at[i,'MEIO_TRANSMISSAO'] == 'ENLACE DE RADIO':
                            nuvens.at[i,'CAPACITY_NUVEM'] = 10
                    nuvens.at[i,'CAPACITY_NUVEM'] = aux_funil.BANDA.values[0]
                    if (v.TECNOLOGIA == 'GPON MOVEL') & (nuvens.at[i,'CAPACITY_NUVEM'] > 200):
                        nuvens.at[i,'CAPACITY_NUVEM'] = 200

                else:
                    nuvens.at[i,'CENTRO_ROTEAMENTO'] = v.ESTACAO_ENTREGA
                    if nuvens.at[i,'REDE'] == 'CORTE CAPACIDADE-BANDA':
                        nuvens.at[i,'CAPACITY_NUVEM'] = 10
                    elif 'CORTE PLANEJAMENTO REGIONAL' in nuvens.at[i,'REDE']:
                        nuvens.at[i,'CAPACITY_NUVEM'] = 0
                    elif nuvens.at[i,'SITUACAO'] == 'ESGOTADA':
                        nuvens.at[i,'CAPACITY_NUVEM'] = 0
                    elif nuvens.at[i,'SITUACAO'] == 'CONCLUIDA':
                        if nuvens.at[i,'MEIO_TRANSMISSAO'] == 'REDE OPTICA':
                            nuvens.at[i,'CAPACITY_NUVEM'] = 100
                        elif nuvens.at[i,'MEIO_TRANSMISSAO'] == 'ENLACE DE RADIO':
                            nuvens.at[i,'CAPACITY_NUVEM'] = 10
                    else:
                        nuvens.at[i,'CAPACITY_NUVEM'] = 0
                    
                if nuvens.at[i,'CAPACITY_NUVEM'] == 0:
                    nuvens.at[i,'CENTRO_ROTEAMENTO'] = v.ESTACAO_ENTREGA
                    if nuvens.at[i,'REDE'] == 'CORTE CAPACIDADE-BANDA':
                        nuvens.at[i,'CAPACITY_NUVEM'] = 10
                    elif 'CORTE PLANEJAMENTO REGIONAL' in nuvens.at[i,'REDE']:
                        nuvens.at[i,'CAPACITY_NUVEM'] = 0
                    elif nuvens.at[i,'SITUACAO'] == 'ESGOTADA':
                        nuvens.at[i,'CAPACITY_NUVEM'] = 0
                    elif nuvens.at[i,'SITUACAO'] == 'CONCLUIDA':
                        if nuvens.at[i,'MEIO_TRANSMISSAO'] == 'REDE OPTICA':
                            nuvens.at[i,'CAPACITY_NUVEM'] = 100
                        elif nuvens.at[i,'MEIO_TRANSMISSAO'] == 'ENLACE DE RADIO':
                            nuvens.at[i,'CAPACITY_NUVEM'] = 10
            
nuvens['CAPACITY_NUVEM'] = nuvens['CAPACITY_NUVEM'].fillna(0)

resumosoe['BANDA_ABORDADO'] = 0

for index, value in resumosoe.iterrows():
    aux_capacity = capacity[(capacity.NUVEM_ABORDADO == value.FACILIDADE_ABORDADO) & (capacity.ESTACAO_ENTREGA == value.ESTACAO_ENTREGA_ABORDADO)]
    aux_capacity_fixa = capacity_fixa[(capacity_fixa.FACILIDADE == value.FACILIDADE_ABORDADO) & (capacity_fixa.SIGLA_EMBRATEL == value.ESTACAO_ENTREGA_ABORDADO)]
    if value.FACILIDADE_ABORDADO == 'FOetherNET':
        resumosoe.at[index,'BANDA_ABORDADO'] = 1000
    elif value.FACILIDADE_ABORDADO == 'FO SDH':
        resumosoe.at[index,'BANDA_ABORDADO'] = 0
    elif len(aux_capacity_fixa) > 0:
        resumosoe.at[index,'BANDA_ABORDADO'] = aux_capacity_fixa.TOTAL.values[0]
    elif len(aux_capacity) > 0:
        resumosoe.at[index,'BANDA_ABORDADO'] = aux_capacity.CAPACITY_MB.values[0]


for index, value in sevs_tratar.iterrows():
    if value.VELOCIDADE[-4] == 'M':
        sevs_tratar.at[index,'VEL'] = int(value.VELOCIDADE[:-4])
    elif value.VELOCIDADE[-4] == 'G':
        sevs_tratar.at[index,'VEL'] = int(value.VELOCIDADE[:-4]) * 1000
    elif value.VELOCIDADE[-4] == 'K':
        sevs_tratar.at[index,'VEL'] = int(value.VELOCIDADE[:-4]) / 1000

sevs_tratar['TRATADO'] = ''

sevs_tratar['FACILIDADE_ABORDADO'] = ''

## ESCOLHA DO ACESSO ABORDADO
for index, value in sevs_tratar.iterrows():
    aux_resumosoe = resumosoe[resumosoe.SEV == value.SEV]
    if len(aux_resumosoe) > 0:
        if (aux_resumosoe.FACILIDADE_ABORDADO.values[0] == 'FO GPON ETH') & (value.VEL <= 202):
            if (aux_resumosoe.BANDA_ABORDADO.values[0] >= value.VEL):
                sevs_tratar.at[index,'FACILIDADE_ABORDADO'] = aux_resumosoe.FACILIDADE_ABORDADO.values[0]
                sevs_tratar.at[index,'FACILIDADE_ACESSO'] = 'FO_GPON_ETH'
                sevs_tratar.at[index,'ESTACAO_ENTREGA_ACESSO'] = aux_resumosoe.ESTACAO_ENTREGA_ABORDADO.values[0]
                sevs_tratar.at[index,'TECNOLOGIA_ACESSO'] = 'GPON FIXA'
                sevs_tratar.at[index,'CENTRO_ROTEAMENTO'] = aux_resumosoe.ESTACAO_ENTREGA_ABORDADO.values[0]
                sevs_tratar.at[index,'TRATADO'] = 'X'
                ########################### VERIFICAR COM LONA PARA REMOVER SDH DOS ABORDADOS AUTOMATICOS
        # elif (aux_resumosoe.FACILIDADE_ABORDADO.values[0] == 'FO SDH') & (value.VEL <= 100):
        #     if (aux_resumosoe.BANDA_ABORDADO.values[0] >= value.VEL):
        #         sevs_tratar.at[index,'FACILIDADE_ABORDADO'] = aux_resumosoe.FACILIDADE_ABORDADO.values[0]
        #         sevs_tratar.at[index,'FACILIDADE_ACESSO'] = 'FO_SDH'
        #         sevs_tratar.at[index,'ESTACAO_ENTREGA_ACESSO'] = aux_resumosoe.ESTACAO_ENTREGA_ABORDADO.values[0]
        #         sevs_tratar.at[index,'TECNOLOGIA_ACESSO'] = 'SDH'
        #         sevs_tratar.at[index,'CENTRO_ROTEAMENTO'] = aux_resumosoe.ESTACAO_ENTREGA_ABORDADO.values[0]
        #         sevs_tratar.at[index,'TRATADO'] = 'X'
        elif (aux_resumosoe.FACILIDADE_ABORDADO.values[0] != 'FO GPON ETH') & (aux_resumosoe.FACILIDADE_ABORDADO.values[0] != 'FO SDH'):
            if (aux_resumosoe.BANDA_ABORDADO.values[0] >= value.VEL):
                sevs_tratar.at[index,'FACILIDADE_ABORDADO'] = aux_resumosoe.FACILIDADE_ABORDADO.values[0]
                if aux_resumosoe.FACILIDADE_ABORDADO.values[0] == 'FO EDD ETH':
                    sevs_tratar.at[index,'FACILIDADE_ACESSO'] = 'FO_EDD_ETH'
                    sevs_tratar.at[index,'TECNOLOGIA_ACESSO'] = 'FO EDD FIXA'
                elif aux_resumosoe.FACILIDADE_ABORDADO.values[0] == 'FOetherNET':
                    sevs_tratar.at[index,'FACILIDADE_ACESSO'] = 'FOETHERNET'
                    sevs_tratar.at[index,'TECNOLOGIA_ACESSO'] = 'FO EDD NET'
                sevs_tratar.at[index,'ESTACAO_ENTREGA_ACESSO'] = aux_resumosoe.ESTACAO_ENTREGA_ABORDADO.values[0]
                sevs_tratar.at[index,'CENTRO_ROTEAMENTO'] = aux_resumosoe.ESTACAO_ENTREGA_ABORDADO.values[0]
                sevs_tratar.at[index,'TRATADO'] = 'X'



for index, value in sevs_tratar[sevs_tratar.TRATADO == 'X'].iterrows():
    if value.SERVICO in ['LAN - LAN EPL','LAN - LAN EPL MEF']:
        sevs_tratar.at[index,'TRATADO'] = ''
        sevs_tratar.at[index,'FACILIDADE_ABORDADO'] = ''
    elif value.SERVICO in ['EIN - TRANSMUX CIRCUITO', 'EIN - TRANSMUX REDE']:
        if value.FACILIDADE_ABORDADO != 'FO SDH':
            sevs_tratar.at[index,'TRATADO'] = ''
            sevs_tratar.at[index,'FACILIDADE_ABORDADO'] = ''

nuvens.NOME_NUVEM = nuvens.NOME_NUVEM.replace(' ','')



for i, v in nuvens.iterrows():
    if v.NOME_NUVEM == '':
        nuvens.at[i,'NOME_NUVEM'] = v.ESTACAO_ENTREGA

for index,value in resumosoe.iterrows():
    for i,v in facilidades.iterrows():
        if 'NUVEM: /' in value[f'{v.FACILIDADE}_INFORMACAO']:
            resumosoe.at[index,f'{v.FACILIDADE}_INFORMACAO'] = value[f'{v.FACILIDADE}_INFORMACAO'].replace('NUVEM: /','')
        elif 'NUVEM:  / ' in value[f'{v.FACILIDADE}_INFORMACAO']:
            resumosoe.at[index,f'{v.FACILIDADE}_INFORMACAO'] = value[f'{v.FACILIDADE}_INFORMACAO'].replace('NUVEM:  / ','')
        elif 'NUVEM: ' in value[f'{v.FACILIDADE}_INFORMACAO']:
            resumosoe.at[index,f'{v.FACILIDADE}_INFORMACAO'] = value[f'{v.FACILIDADE}_INFORMACAO'].replace('NUVEM: ','')
        
        if resumosoe.at[index,f'{v.FACILIDADE}_INFORMACAO'] == '':
            resumosoe.at[index,f'{v.FACILIDADE}_INFORMACAO'] = resumosoe.at[index,f'{v.FACILIDADE}_ESTACAO_ENTREGA']


for index, value in sevs_tratar[sevs_tratar.SEV == 7814855].iterrows():
    aux_resumosoe = resumosoe[resumosoe.SEV == value.SEV]
    for i,v in facilidades.iterrows():
        
        if sevs_tratar.at[index,'TRATADO'] != 'X':
            
            if aux_resumosoe[f'{v.FACILIDADE}'].values[0] == 'Viável':
                print(v.FACILIDADE)
                if v.VERIFICA_CAPACITY == 'N':
                    if v.FACILIDADE == 'HFC_BSOD':
                        if 'HP GED' in aux_resumosoe[f'{v.FACILIDADE}_INFORMACAO'].values[0]:
                            sevs_tratar.at[index,'FACILIDADE_ACESSO'] = v.FACILIDADE
                            sevs_tratar.at[index,'NUVEM_ACESSO'] = aux_resumosoe[f'{v.FACILIDADE}_INFORMACAO'].values[0]
                            sevs_tratar.at[index,'ESTACAO_ENTREGA_ACESSO'] = ''
                            sevs_tratar.at[index,'TECNOLOGIA_ACESSO'] = 'HFC BSOD'
                            sevs_tratar.at[index,'TRATADO'] = 'X'
                            break
                        else:
                            sevs_tratar.at[index,'FACILIDADE_ACESSO'] = v.FACILIDADE
                            sevs_tratar.at[index,'NUVEM_ACESSO'] = aux_resumosoe[f'{v.FACILIDADE}_ESTACAO_ENTREGA'].values[0]
                            sevs_tratar.at[index,'ESTACAO_ENTREGA_ACESSO'] = aux_resumosoe[f'{v.FACILIDADE}_INFORMACAO'].values[0].replace('ESTAÇÃO ENTRONCAMENTO:','')
                            sevs_tratar.at[index,'TECNOLOGIA_ACESSO'] = 'HFC BSOD'
                            sevs_tratar.at[index,'TRATADO'] = 'X'
                            break
                    elif v.FACILIDADE == '4G':
                        sevs_tratar.at[index,'FACILIDADE_ACESSO'] = v.FACILIDADE
                        sevs_tratar.at[index,'NUVEM_ACESSO'] = aux_resumosoe[f'{v.FACILIDADE}_INFORMACAO'].values[0]
                        sevs_tratar.at[index,'ESTACAO_ENTREGA_ACESSO'] = ''
                        sevs_tratar.at[index,'TECNOLOGIA_ACESSO'] = 'LTE (4G)'
                        sevs_tratar.at[index,'TRATADO'] = 'X'
                        break
                    elif v.FACILIDADE == 'FO_GPON_RESID_ETH_PRE_VIAVEL':
                        for tec in v.TECNOLOGIA.split('/'):
                            if sevs_tratar.at[index,'TRATADO'] != 'X':
                                for nome_nuvem in aux_resumosoe[f'{v.FACILIDADE}_INFORMACAO'].values[0].split(' / '):
                                    aux_nuvem = nuvens[(nuvens.SEV == value.SEV) & (nuvens.NOME_NUVEM == nome_nuvem) & (nuvens.TECNOLOGIA == tec)]
                                    if len(aux_nuvem) > 0:
                                        if value.VEL <= aux_nuvem.CAPACITY_NUVEM.values[0]:
                                            sevs_tratar.at[index,'FACILIDADE_ACESSO'] = v.FACILIDADE
                                            sevs_tratar.at[index,'NUVEM_ACESSO'] = f'FABRICANTE {aux_nuvem.FABRICANTE_OLT.values[0]} CONCENTRADOR OLT {aux_nuvem.CONCENTRADOR_OLT.values[0]}'
                                            sevs_tratar.at[index,'ESTACAO_ENTREGA_ACESSO'] = aux_resumosoe[f'{v.FACILIDADE}_ESTACAO_ENTREGA'].values[0] 
                                            sevs_tratar.at[index,'TECNOLOGIA_ACESSO'] = tec
                                            sevs_tratar.at[index,'TRATADO'] = 'X'
                                            break
                    elif v.FACILIDADE == 'FO_XGSPON_RESID_ETH':
                        for tec in v.TECNOLOGIA.split('/'):
                            if sevs_tratar.at[index,'TRATADO'] != 'X':
                                for nome_nuvem in aux_resumosoe[f'{v.FACILIDADE}_INFORMACAO'].values[0].split(' / '):
                                    aux_nuvem = nuvens[(nuvens.SEV == value.SEV) & (nuvens.NOME_NUVEM == nome_nuvem) & (nuvens.TECNOLOGIA == tec)]
                                    if len(aux_nuvem) > 0:
                                        if value.VEL <= aux_nuvem.CAPACITY_NUVEM.values[0]:
                                            sevs_tratar.at[index,'FACILIDADE_ACESSO'] = v.FACILIDADE
                                            sevs_tratar.at[index,'NUVEM_ACESSO'] = f'FABRICANTE {aux_nuvem.FABRICANTE_OLT.values[0]} CONCENTRADOR OLT {aux_nuvem.CONCENTRADOR_OLT.values[0]}'
                                            sevs_tratar.at[index,'ESTACAO_ENTREGA_ACESSO'] = aux_resumosoe[f'{v.FACILIDADE}_ESTACAO_ENTREGA'].values[0] 
                                            sevs_tratar.at[index,'TECNOLOGIA_ACESSO'] = tec
                                            sevs_tratar.at[index,'TRATADO'] = 'X'
                                            break
                    elif v.FACILIDADE == 'FO_GPON_RESID_ETH':
                        for tec in v.TECNOLOGIA.split('/'):
                            if sevs_tratar.at[index,'TRATADO'] != 'X':
                                for nome_nuvem in aux_resumosoe[f'{v.FACILIDADE}_INFORMACAO'].values[0].split(' / '):
                                    aux_nuvem = nuvens[(nuvens.SEV == value.SEV) & (nuvens.NOME_NUVEM == nome_nuvem) & (nuvens.TECNOLOGIA == tec)]
                                    if len(aux_nuvem) > 0:
                                        if value.VEL <= aux_nuvem.CAPACITY_NUVEM.values[0]:
                                            sevs_tratar.at[index,'FACILIDADE_ACESSO'] = v.FACILIDADE
                                            sevs_tratar.at[index,'NUVEM_ACESSO'] = f'FABRICANTE {aux_nuvem.FABRICANTE_OLT.values[0]} CONCENTRADOR OLT {aux_nuvem.CONCENTRADOR_OLT.values[0]}'
                                            sevs_tratar.at[index,'ESTACAO_ENTREGA_ACESSO'] = aux_resumosoe[f'{v.FACILIDADE}_ESTACAO_ENTREGA'].values[0] 
                                            sevs_tratar.at[index,'TECNOLOGIA_ACESSO'] = tec
                                            sevs_tratar.at[index,'TRATADO'] = 'X'
                                            break

                    elif (v.FACILIDADE == 'SATELITE_BANDA_KA') | (v.FACILIDADE == 'SATELITE_BANDA_KU'):
                        sevs_tratar.at[index,'FACILIDADE_ACESSO'] = v.FACILIDADE
                        sevs_tratar.at[index,'NUVEM_ACESSO'] = aux_resumosoe.TERCEIROS_ETH_INFORMACAO.values[0]
                        sevs_tratar.at[index,'ESTACAO_ENTREGA_ACESSO'] = 'RJO AM'
                        sevs_tratar.at[index,'TECNOLOGIA_ACESSO'] = v.TECNOLOGIA
                        sevs_tratar.at[index,'TRATADO'] = 'X'
                        break
                else:
                    if v.FACILIDADE == 'TERCEIROS_ETH':
                        
                        sevs_tratar.at[index,'FACILIDADE_ACESSO'] = v.FACILIDADE
                        sevs_tratar.at[index,'NUVEM_ACESSO'] = aux_resumosoe.TERCEIROS_ETH_INFORMACAO.values[0].split('PROPRIETÁRIO ')[-1]
                        sevs_tratar.at[index,'ESTACAO_ENTREGA_ACESSO'] = aux_resumosoe.TERCEIROS_ETH_ESTACAO_ENTREGA.values[0].split(' / ')[-1]
                        sevs_tratar.at[index,'TECNOLOGIA_ACESSO'] = 'TERCEIRO ETH'
                        sevs_tratar.at[index,'CENTRO_ROTEAMENTO'] = aux_resumosoe.TERCEIROS_ETH_ESTACAO_ENTREGA.values[0].split(' / ')[-1]
                        sevs_tratar.at[index,'TRATADO'] = 'X'
                        
                    else:
                        for tec in v.TECNOLOGIA.split('/'):
                            print(v.FACILIDADE)
                            print(tec)
                            if sevs_tratar.at[index,'TRATADO'] != 'X':
                                for nome_nuvem in aux_resumosoe[f'{v.FACILIDADE}_INFORMACAO'].values[0].split(' / '):
                                    aux_nuvem = nuvens[(nuvens.SEV == value.SEV) & (nuvens.NOME_NUVEM == nome_nuvem.replace(':','')) & (nuvens.TECNOLOGIA == tec)]
                                    if len(aux_nuvem) > 0:
                                        if (value.SERVICO == 'LAN - LAN EPL MEF') & (value.VEL <= aux_nuvem.CAPACITY_NUVEM.values[0]):
                                            if 'EPL MEF - NOK' not in aux_nuvem.OBSERVACAO.values[0]:
                                                sevs_tratar.at[index,'FACILIDADE_ACESSO'] = v.FACILIDADE
                                                if tec in v.CONSULTA_FUNIL.split('/'):
                                                    sevs_tratar.at[index,'NUVEM_ACESSO'] = f'NUVEM {aux_nuvem.NOME_NUVEM.values[0]} SITE {aux_nuvem.SIGLA_ESTACAO_CLARO.values[0]}'
                                                else:
                                                    sevs_tratar.at[index,'NUVEM_ACESSO'] = aux_nuvem.NOME_NUVEM.values[0]
                                                sevs_tratar.at[index,'ESTACAO_ENTREGA_ACESSO'] = aux_nuvem.ESTACAO_ENTREGA.values[0]
                                                sevs_tratar.at[index,'CENTRO_ROTEAMENTO'] = aux_nuvem.CENTRO_ROTEAMENTO.values[0]
                                                sevs_tratar.at[index,'TECNOLOGIA_ACESSO'] = tec
                                                sevs_tratar.at[index,'TRATADO'] = 'X'
                                                break

                                        if (value.SERVICO != 'LAN - LAN EPL MEF') & (value.VEL <= aux_nuvem.CAPACITY_NUVEM.values[0]):
                                            sevs_tratar.at[index,'FACILIDADE_ACESSO'] = v.FACILIDADE
                                            if tec in v.CONSULTA_FUNIL.split('/'):
                                                sevs_tratar.at[index,'NUVEM_ACESSO'] = f'NUVEM {aux_nuvem.NOME_NUVEM.values[0]} SITE {aux_nuvem.SIGLA_ESTACAO_CLARO.values[0]}'
                                            else:
                                                sevs_tratar.at[index,'NUVEM_ACESSO'] = aux_nuvem.NOME_NUVEM.values[0]
                                            sevs_tratar.at[index,'ESTACAO_ENTREGA_ACESSO'] = aux_nuvem.ESTACAO_ENTREGA.values[0]
                                            sevs_tratar.at[index,'CENTRO_ROTEAMENTO'] = aux_nuvem.CENTRO_ROTEAMENTO.values[0]
                                            sevs_tratar.at[index,'TECNOLOGIA_ACESSO'] = tec
                                            sevs_tratar.at[index,'TRATADO'] = 'X'
                                            break

                    
            if aux_resumosoe[f'{v.FACILIDADE}'].values[0] == '%Disponibilidade não atende ao desejado':
                if (v.FACILIDADE == 'SATELITE_BANDA_KA') | (v.FACILIDADE == 'SATELITE_BANDA_KU'):
                    sevs_tratar.at[index,'FACILIDADE_ACESSO'] = v.FACILIDADE
                    sevs_tratar.at[index,'NUVEM_ACESSO'] = aux_resumosoe.TERCEIROS_ETH_INFORMACAO.values[0]
                    sevs_tratar.at[index,'ESTACAO_ENTREGA_ACESSO'] = 'RJO AM'
                    sevs_tratar.at[index,'TECNOLOGIA_ACESSO'] = v.TECNOLOGIA
                    sevs_tratar.at[index,'TRATADO'] = 'X'

sevs_tratar.FACILIDADE_ACESSO = sevs_tratar.FACILIDADE_ACESSO.fillna('')
sevs_tratar.CENTRO_ROTEAMENTO = sevs_tratar.CENTRO_ROTEAMENTO.fillna('')


for i,v in valores_ethernet.iterrows():
    try:
        if v.velocidade[-1] == 'M':
            valores_ethernet.at[i,'VEL'] = int(v.velocidade[:-1])
        elif v.velocidade[-1] == 'G':
            valores_ethernet.at[i,'VEL'] = float(v.velocidade[:-1].replace(',','.')) * 1000
        elif v.velocidade[-1] == 'K':
            valores_ethernet.at[i,'VEL'] = float(v.velocidade[:-1].replace(',','.')) / 1000
    except:
        pass

valores_ethernet = valores_ethernet.fillna(0)

for index, value in sevs_tratar[(sevs_tratar.FACILIDADE_ACESSO == 'TERCEIROS_ETH')].iterrows():
    melhor_provedor = ''
    melhor_instal = 0
    melhor_mensal = 0
    custo_mensalizado = 0
    for p in value.NUVEM_ACESSO.split(' / '):
        aux_provedor = provedor_ethernet[(provedor_ethernet.PROVEDOR == p) & (provedor_ethernet.SIGLA_LOC == value.CNL) & (provedor_ethernet.UF == value.UF)]
        if len(aux_provedor) > 0:
            aux_status = status[(status.provedor == aux_provedor.PROVEDOR.values[0]) & (status.uf == aux_provedor.UF.values[0])]
            if len(aux_status) > 0:
                if (aux_status.status.values[0] != 'BLOQUEADO') & (aux_status.status.values[0] != 'AEROPORTO'):
                    aux_valores = valores_ethernet[(valores_ethernet.contrato == aux_provedor.CONTRATO.values[0]) & (valores_ethernet.VEL == value.VEL) & (valores_ethernet.tempo_contrato == '24 meses')].sort_values(by='VEL', ascending=True)
                    
                    if len(aux_valores) > 0:
                        if melhor_provedor == '':
                            melhor_provedor = p
                            melhor_instal = valores_ethernet.instalacao.values[0]
                            melhor_mensal = valores_ethernet.mensalidade.values[0]
                            custo_mensalizado = (melhor_instal / 24) + melhor_mensal 
                        else:
                            if custo_mensalizado > ((valores_ethernet.instalacao.values[0] / 24) + valores_ethernet.mensalidade.values[0]):
                                melhor_provedor = p
                                melhor_instal = valores_ethernet.instalacao.values[0]
                                melhor_mensal = valores_ethernet.mensalidade.values[0]
                                custo_mensalizado = (melhor_instal / 24) + melhor_mensal
                    else:
                        aux_valores = valores_ethernet[(valores_ethernet.contrato == provedor_ethernet.CONTRATO.values[0]) & (valores_ethernet.VEL >= value.VEL) & (valores_ethernet.tempo_contrato == '24 meses')].sort_values(by='VEL', ascending=True)
                        if len(aux_valores) > 0:
                            if melhor_provedor == '':
                                melhor_provedor = p
                                melhor_instal = valores_ethernet.instalacao.values[0]
                                melhor_mensal = valores_ethernet.mensalidade.values[0]
                                custo_mensalizado = (melhor_instal / 24) + melhor_mensal 
                            else:
                                if custo_mensalizado > ((valores_ethernet.instalacao.values[0] / 24) + valores_ethernet.mensalidade.values[0]):
                                    melhor_provedor = p
                                    melhor_instal = valores_ethernet.instalacao.values[0]
                                    melhor_mensal = valores_ethernet.mensalidade.values[0]
                                    custo_mensalizado = (melhor_instal / 24) + melhor_mensal     
    if melhor_provedor != '':
        sevs_tratar.at[index,'PROVEDOR_ESCOLHA'] = melhor_provedor
        sevs_tratar.at[index,'PROVEDOR_INSTALACAO'] = melhor_instal
        sevs_tratar.at[index,'PROVEDOR_MENSALIDADE'] = melhor_mensal
    else:
        sevs_tratar.loc[index, ['TRATADO','FACILIDADE_ACESSO','NUVEM_ACESSO','ESTACAO_ENTREGA_ACESSO','TECNOLOGIA_ACESSO','CENTRO_ROTEAMENTO']] = ''

for index, value in sevs_tratar.iterrows():
    if value.SERVICO != 'VPE - VIP BSOD LIGHT':
        if value.VEL > 102:
            if value.FACILIDADE_ACESSO in facilidades[facilidades.VERIFICA_CAPACITY == 'S'].FACILIDADE.tolist():

                aux_municipio_estacao = municipio_estacao[municipio_estacao.ESTACAO == value.CENTRO_ROTEAMENTO]
                if len(aux_municipio_estacao) > 0:
                    sevs_tratar.at[index,'BBIP'] = f'{aux_municipio_estacao.MUNICIPIO.values[0]}|{aux_municipio_estacao.ESTACAO.values[0]}'
                else:
                    aux_estacao_uf= estacoes_uf[estacoes_uf.UF == value.UF]
                    sevs_tratar.at[index,'BBIP'] = f'{aux_estacao_uf.MUNICIPIO.values[0]}|{aux_estacao_uf.ESTACAO.values[0]}'
try:    
    sevs_tratar.BBIP = sevs_tratar.BBIP.fillna('')
except:
    sevs_tratar['BBIP'] = ''

for index, value in sevs_tratar.iterrows():
    sevs_tratar.at[index,'PROTOCOLO_ATENDIMENTO'] = resultado[resultado.SEV == value.SEV].PROTOCOLO.values[0]

TERCEIROS_ETH


In [45]:
nuvens[nuvens.SEV == 7814855]

,SEV,OBJECTID,ESTACAO_ENTREGA,UF,SIGLA_LOC,NOME_NUVEM,REDE,TECNOLOGIA,MEIO_TRANSMISSAO,SITUACAO,ALT_NUVEM,PADRAO_PON,PROPRIETARIO,OBSERVACAO,ORIGEM,DATA_ATUALIZACAO,DATA_PREVISAO,VEL_MAX_VIABILIDADE,VEL_MAX_SEV_AUTOMATICA,MOTIVO,OPERADORA,NUMERO_OPERACIONAL,SIGLA_ESTACAO_CLARO,CODIGO_DESCARGA_CSL,SIGLA_ESTACAO_RESID,TIPO_INFRA,TIPO,STATUS,DONO,ROTEADORES_QTD,TX,FABRICANTE_OLT,ABRANGENCIA,CONCENTRADOR_OLT,CENTRO_ROTEAMENTO,CAPACITY_NUVEM
91,7814855,7856559,CXY MQ,AM,CXY,AMEBT11 - NUVEM PRE VIAVEL,ATEND.CLARO - PRE (CORTE PLANEJAMENTO REGIONAL),FO EDD PV,REDE OPTICA,AVALIAR CAPACIDADE,,,CLARO BRASIL,EPL MEF - OK/ CORTE REGIONAL - NECESSARIO CONS...,MAPINFO,3/26/2025 103015 PM,,,,,,,AMEBT11,,,GREENFIELD,FIXA,MOBILIZADA,CLARO,7250-IXR-R6/7705-SAR8,FO,,,,CXY MQ,500.0
92,7814855,7829284,MNS EM,AM,CXY,AMCXY01 - NUVEM DIRETO CLARO,ATEND.CLARO - DC (CORTE PLANEJAMENTO REGIONAL),FO EDD DC,REDE OPTICA,AVALIAR CAPACIDADE,,,CLARO BRASIL,EPL MEF - OK/ CORTE REGIONAL - NECESSARIO CONS...,MAPINFO,3/25/2025 101100 PM,,,,,,,AMCXY01,,,GREENFIELD,FIXA,MOBILIZADA,COMPARTILHADO,,FO,,,,MNS EM,10.0


In [51]:
resumosoe[resumosoe.SEV == 7814855]

,ID,SEV,GEOCODE_OK,LATITUDE,LONGITUDE,DEGRAU_PARA_TERCEIROS,RESTRIÇÃO,4G,4G_INFORMACAO,4G_ESTACAO_ENTREGA,4G_DISPONIBILIDADE,HFC_BSOD,HFC_BSOD_INFORMACAO,HFC_BSOD_ESTACAO_ENTREGA,HFC_BSOD_DISPONIBILIDADE,FO_GPON_RESID_ETH,FO_GPON_RESID_ETH_INFORMACAO,FO_GPON_RESID_ETH_ESTACAO_ENTREGA,FO_GPON_RESID_ETH_DISPONIBILIDADE,FO_GPON_ETH,FO_GPON_ETH_INFORMACAO,FO_GPON_ETH_ESTACAO_ENTREGA,FO_GPON_ETH_DISPONIBILIDADE,FOETHERNET,FOETHERNET_INFORMACAO,FOETHERNET_ESTACAO_ENTREGA,FOETHERNET_DISPONIBILIDADE,FO_EDD_ETH,FO_EDD_ETH_INFORMACAO,FO_EDD_ETH_ESTACAO_ENTREGA,FO_EDD_ETH_DISPONIBILIDADE,FO_EDD_DIRETO_CLARO,FO_EDD_DIRETO_CLARO_INFORMACAO,FO_EDD_DIRETO_CLARO_ESTACAO_ENTREGA,FO_EDD_DIRETO_CLARO_DISPONIBILIDADE,FO_EDD_CLARO_PRE_VIAVEL,FO_EDD_CLARO_PRE_VIAVEL_INFORMACAO,FO_EDD_CLARO_PRE_VIAVEL_ESTACAO_ENTREGA,FO_EDD_CLARO_PRE_VIAVEL_DISPONIBILIDADE,FO_SDH,FO_SDH_INFORMACAO,FO_SDH_ESTACAO_ENTREGA,FO_SDH_DISPONIBILIDADE,ADE_SDH,ADE_SDH_INFORMACAO,ADE_SDH_ESTACAO_ENTREGA,ADE_SDH_DISPONIBILIDADE,SATELITE_BANDA_KA,SATELITE_BANDA_KA_INFORMACAO,SATELITE_BANDA_KA_ESTACAO_ENTREGA,SATELITE_BANDA_KA_DISPONIBILIDADE,SATELITE_BANDA_KU,SATELITE_BANDA_KU_INFORMACAO,SATELITE_BANDA_KU_ESTACAO_ENTREGA,SATELITE_BANDA_KU_DISPONIBILIDADE,TERCEIROS_ETH,TERCEIROS_ETH_INFORMACAO,TERCEIROS_ETH_ESTACAO_ENTREGA,TERCEIROS_ETH_DISPONIBILIDADE,FO_GPON_RESID_ETH_PRE_VIAVEL,FO_GPON_RESID_ETH_PRE_VIAVEL_INFORMACAO,FO_GPON_RESID_ETH_PRE_VIAVEL_ESTACAO_ENTREGA,FO_GPON_RESID_ETH_PRE_VIAVEL_DISPONIBILIDADE,FO_XGSPON_RESID_ETH,FO_XGSPON_RESID_ETH_INFORMACAO,FO_XGSPON_RESID_ETH_ESTACAO_ENTREGA,FO_XGSPON_RESID_ETH_DISPONIBILIDADE,RADIO_IP,RADIO_IP_INFORMACAO,RADIO_IP_ESTACAO_ENTREGA,RADIO_IP_DISPONIBILIDADE,ID_ACESSO_ABORDADO,FACILIDADE_ABORDADO,TECNOLOGIA_ABORDADO,ESTACAO_ENTREGA_ABORDADO,BANDA_ABORDADO
28,29,7814855,SIM,-3.82501715,-60.3632006,,NÃO,Inviável,,,0%,Inviável,,,0%,Inviável,,,0%,Inviável,,,0%,Inviável,,,0%,Inviável,,,0%,Nuvem Avaliar Capacidade,AMCXY01 - NUVEM DIRETO CLARO,MNS EM,"99,957%",Nuvem Avaliar Capacidade,AMEBT11 - NUVEM PRE VIAVEL,CXY MQ,"99,957%",Inviável,,,0%,Inviável,,,0%,Inviável,,,0%,Inviável,,,0%,Viável,CAREIRO - PROPRIETÁRIO SITELBRA,MNS EM,0%,Inviável,,,0%,Inviável,,,0%,End. não localizado,,,,,,,,0.0


In [10]:
nuvens[nuvens.SEV == 7755753]

,SEV,OBJECTID,ESTACAO_ENTREGA,UF,SIGLA_LOC,NOME_NUVEM,REDE,TECNOLOGIA,MEIO_TRANSMISSAO,SITUACAO,ALT_NUVEM,PADRAO_PON,PROPRIETARIO,OBSERVACAO,ORIGEM,DATA_ATUALIZACAO,DATA_PREVISAO,VEL_MAX_VIABILIDADE,VEL_MAX_SEV_AUTOMATICA,MOTIVO,OPERADORA,NUMERO_OPERACIONAL,SIGLA_ESTACAO_CLARO,CODIGO_DESCARGA_CSL,SIGLA_ESTACAO_RESID,TIPO_INFRA,TIPO,STATUS,DONO,ROTEADORES_QTD,TX,FABRICANTE_OLT,ABRANGENCIA,CONCENTRADOR_OLT,CAPACITY_NUVEM,CENTRO_ROTEAMENTO
0,7755753,7749305,SLS RDF,MA,SLS,SLS RDF_SLS RDF_ANEL MASLS24A LADO 1_SLS,SLS RDF,SDH,REDE OPTICA,CONCLUIDA,,,CLARO BRASIL,,MAPINFO,2/21/2025 100139 PM,,,,,,0,,,,,,,,,,,,,622.0,SLS US
1,7755753,7787472,SLS C30,MA,SLS,MASLS30 - NUVEM DIRETO CLARO,ATEND. CLARO - DC,FO EDD DC,REDE OPTICA,CONCLUIDA,,,CLARO BRASIL,EPL MEF - NOK,MAPINFO,2/27/2025 124846 AM,,,,,,,MASLS30,,,GREENFIELD,FIXA,MOBILIZADA,COMPARTILHADO,,FO,,,,500.0,SLS C30
2,7755753,6767095,HE_BELEM,MA,SLS,PSLAFA,,FO EDD NET,REDE OPTICA,CONCLUIDA,,,CLARO BRASIL,,MAPINFO,2/8/2024 54036 PM,,,,,,,,,HE,,,,,,,,,,961.0,BLM QB
3,7755753,6767095,HE_BELEM,MA,SLS,PSLAFA,,GPON NET,REDE OPTICA,CONCLUIDA,,,CLARO BRASIL,,MAPINFO,2/8/2024 54036 PM,,,,,,,,,HE,,,,,,,,,,200.0,BLM QB
4,7755753,7749360,SLS US,MA,SLS,PSLAFA,VIAVEL,GPON RES RESID PV,REDE OPTICA,CONCLUIDA,,GPON,CLARO BRASIL,GPON RESIDENCIAL PRE-VIAVEL,MAPINFO,2/21/2025 100230 PM,,,,,,0,MASLSC0,,HE_BELEM,,,,,,,,,SLSPSLACOLT01,500.0,NaN


In [18]:
sevs_tratar[sevs_tratar.FACILIDADE_ACESSO != '']

,ID_ANALISE,ID_TRIAGEM,CAIXA,SEV,CLIENTE,CLIENTE_FINAL,VELOCIDADE,CNL,DATA_DE_INCLUSAO,DATA_DE_TRIAGEM,DATA_FECHAMENTO_TEIA,DIAS,SOLICITANTE,SOLUCIONADOR,PRIORIZAR,FINALIZADO,TIPO_LOGRADOURO,NOME_DO_LOGRADOURO,NUMERO,COMPLEMENTO,BAIRRO,CIDADE,UF,ESTADO,REGIONAL,CEP,SERVICO,VELOCIDADE_SERV,QTDE_CIRCUITOS,PABX_VIRTUAL,OBS_SOLICITANTE,PONTA_A,LATITUDE,LONGITUDE,CODGEO,INDGEO,FASE,SUBFASE,OBS_PENDENCIA,STATUS_FECHAMENTO,FACILIDADE,PROTOCOLO_GAIA,ABORDADO,FACILIDADE_ABORDADA,PRAZO,PRAZO_MANUAL,JUSTIFICATIVA_PRAZO_MANUAL,DISPONIBILIDADE,RESERVA_BACKBONE_IP,PROTOCOLO_IP,RESERVA_BACKBONE_TELEFONIA,PROTOCOLO_TELEFONIA,ESTACAO_DE_ENTREGA,EMPRESA_FORNECEDORA,CUSTO_TOTAL,MENSALIDADE_TERCEIRO,TAXA_DE_INSTALACAO_TERCEIRO,CUSTO_ACESSO,CUSTO_ACESSO_EQUIPAMENTO,CUSTO_PLANTA_EQUIPAMENTO,CUSTO_PLANTA_REDE_EXTERNA,CUSTO_EXCLUSIVO_ACESSO,RESPOSTA_SEV,ACAO,PROJETO,VEL,TRATADO,FACILIDADE_ABORDADO,FACILIDADE_ACESSO,ESTACAO_ENTREGA_ACESSO,TECNOLOGIA_ACESSO,CENTRO_ROTEAMENTO,NUVEM_ACESSO,PROVEDOR_ESCOLHA,PROVEDOR_INSTALACAO,PROVEDOR_MENSALIDADE,BBIP,PROTOCOLO_ATENDIMENTO
0,937755,1323652,REDE 46 - MA,7755753,TESTE NELSON,TESTE,200Mbps,SLS,2025-03-17,2025-03-17 11:14:31,NaN,1,PROENCA,NaN,NaN,NaN,AV,DANIEL DE LA TOUCHE,987,NaN,COHAMA,SAO LUIS,MA,MARANHAO,CONO,65074115,BLD - Business Link Direct,200Mbps,1,NaN,Golden Jumper,,"-2,50269795","-44,2399837",NaN,NaN,Viabilidade,Análise,NaN,NaN,NaN,10605488,NAO,NaN,NaN,NaN,NaN,NaN,NAO,NaN,NAO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ativar,PORTFOLIO,200.0,X,FO GPON ETH,FO_GPON_ETH,SLS RES,GPON FIXA,SLS RES,NaN,NaN,NaN,NaN,SAO LUIS|SLS US,10613195
1,937757,1323654,REDE 66 - DF,7755757,DHL LOGISTICS (BRAZIL) LTDA,DHL LOGISTICS (BRAZIL) LTDA,200Mbps,BSA,2025-03-17,2025-03-17 11:14:31,NaN,1,ANGELOB,NaN,NaN,NaN,ROD,BR-040,NaN,NaN,COND CHS ANA MARIA SANTA MARIA,BRASILIA,DF,DISTRITO FEDERAL,CONO,72579250,BLD - Business Link Direct,200Mbps,1,NaN,Serviço/Velocidade SEV Manual,,"-16,0448507","-47,9847294",NaN,NaN,Viabilidade,Análise,NaN,NaN,NaN,10605494,NAO,NaN,NaN,NaN,NaN,NaN,NAO,NaN,NAO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ativar,PORTFOLIO,200.0,X,,TERCEIROS_ETH,GNA RD,TERCEIRO ETH,GNA RD,SITELBRA,SITELBRA,0.0,0.0,GOIANIA|GNA RD,10613197
2,937760,1323657,REDE 66 - DF,7755766,DHL LOGISTICS (BRAZIL) LTDA,DHL LOGISTICS (BRAZIL) LTDA,300Mbps,BSA,2025-03-17,2025-03-17 11:14:32,NaN,1,ANGELOB,NaN,NaN,NaN,ROD,BR-040,NaN,NaN,COND CHS ANA MARIA SANTA MARIA,BRASILIA,DF,DISTRITO FEDERAL,CONO,72579250,BLD - Business Link Direct,300Mbps,1,NaN,Serviço/Velocidade SEV Manual,,"-16,0448507","-47,9847294",NaN,NaN,Viabilidade,Análise,NaN,NaN,NaN,10605601,NAO,NaN,NaN,NaN,NaN,NaN,NAO,NaN,NAO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ativar,PORTFOLIO,300.0,X,,TERCEIROS_ETH,GNA RD,TERCEIRO ETH,GNA RD,SITELBRA,SITELBRA,0.0,0.0,GOIANIA|GNA RD,10613198
5,937802,1323718,SERVICO - REDE 41 - ACESTER-RJ,7755859,TELMEX DO BRASIL S A,TELMEX BRASIL,1Gbps,PTS,2025-03-17,2025-03-17 11:14:38,NaN,1,ADEMIR,NaN,NaN,NaN,R,DO IMPERADOR,288,NaN,CENTRO,PETROPOLIS,RJ,RIO DE JANEIRO,RJ/ES,25620000,BLD - Business Link Direct,1Gbps,1,NaN,Serviço/Velocidade SEV Manual,,"-22,5084321","-43,1726770",NaN,NaN,Viabilidade,Análise,NaN,NaN,NaN,10606386,NAO,NaN,NaN,NaN,NaN,NaN,NAO,NaN,NAO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ativar,PORTFOLIO,1000.0,X,,FO_EDD_ETH,PTS BR,FO EDD FIXA,RJO AM,PTS BR_PTSBR_NUVEMOPT_PTS_16,NaN,NaN,NaN,RIO DE JANEIRO|RJO AM,10613201
6,937805,1323721,REDE 51 - ACESTER,7755862,RTM - REDE DE TELECOMUNICACOES PARA O MERCADO ...,RTM,50Mbps,SPO,2025-03-17,2025-03-17 11:14:38,NaN,1,CMMOTA,NaN,NaN,NaN,R,LIBERO BADARO,377,AN 26,CENTRO,SAO PAULO,SP,SAO PAULO,SP METRO,1009906,UNV - Rede Única de Dados MPLS,50Mbps,1,NaN,Facilidade Terceiros,,"-23,5464700","-46,6362250",NaN,NaN,Viabilidade,Análise,NaN,NaN,NaN,10606404,NAO,NaN,NaN,NaN,NaN,NaN,NAO,NaN,NAO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ativar,PORTFOLIO,50.0,X,,TERCEIROS_ETH,SPO IG,TERCEIRO ETH,SPO IG,WCS / ALGAR TELECOM / UPIX / ASCENTY / BRFIBRA,UPIX,0.0,0.0,,10613

In [283]:
restricao[restricao.SEV == 7617521].TIPO_DE_IMPACTO.tolist()

,ID,SEV,OBJECTID,ESTADO,SIGLA_DA_LOCALIDADE,NOME,TIPO_DE_IMPACTO,TIPO_RESTRICAO,TIPO_DA_AREA,MEIO_TRANS_IMPACTADO,TECNOLOGIA_IMPACTADA,OBSERVACAO,DATA_ATUALIZACAO,ORIGEM,COD_PRO,POSICAO
1,89,7617521,7532383,SP,STS,GONZAGA,PARCIAL,INVIAVEL GPON NET,,REDE OPTICA,GPON NET,REDE ESGOTADA E SEM POSIBILIDADE DE ESPANCAO N...,11/6/2024 102246 PM,MAPINFO,25,1
2,89,7617521,7532457,SP,STS,SHOPPING PARQUE BALNEARIO,TOTAL,AV ANA COSTA 549 - GONZAGA SANTOS - SP 11060-003,SHOPPING,REDE METÁLICA / REDE OPTICA,VIRTUA,INVIAVEL PARA BLC,11/6/2024 102311 PM,MAPINFO,25,2


In [9]:
sevs_tratar[sevs_tratar.SEV.isin([7622693,7622822])]

,ID_ANALISE,ID_TRIAGEM,CAIXA,SEV,CLIENTE,CLIENTE_FINAL,VELOCIDADE,CNL,DATA_DE_INCLUSAO,DATA_DE_TRIAGEM,DATA_FECHAMENTO_TEIA,DIAS,SOLICITANTE,SOLUCIONADOR,PRIORIZAR,FINALIZADO,TIPO_LOGRADOURO,NOME_DO_LOGRADOURO,NUMERO,COMPLEMENTO,BAIRRO,CIDADE,UF,ESTADO,REGIONAL,CEP,SERVICO,VELOCIDADE_SERV,QTDE_CIRCUITOS,PABX_VIRTUAL,OBS_SOLICITANTE,PONTA_A,LATITUDE,LONGITUDE,CODGEO,INDGEO,FASE,SUBFASE,OBS_PENDENCIA,STATUS_FECHAMENTO,FACILIDADE,PROTOCOLO_GAIA,ABORDADO,FACILIDADE_ABORDADA,PRAZO,PRAZO_MANUAL,JUSTIFICATIVA_PRAZO_MANUAL,DISPONIBILIDADE,RESERVA_BACKBONE_IP,PROTOCOLO_IP,RESERVA_BACKBONE_TELEFONIA,PROTOCOLO_TELEFONIA,ESTACAO_DE_ENTREGA,EMPRESA_FORNECEDORA,CUSTO_TOTAL,MENSALIDADE_TERCEIRO,TAXA_DE_INSTALACAO_TERCEIRO,CUSTO_ACESSO,CUSTO_ACESSO_EQUIPAMENTO,CUSTO_PLANTA_EQUIPAMENTO,CUSTO_PLANTA_REDE_EXTERNA,CUSTO_EXCLUSIVO_ACESSO,RESPOSTA_SEV,ACAO,PROJETO,VEL,TRATADO,FACILIDADE_ABORDADO,FACILIDADE_ACESSO,TECNOLOGIA_ACESSO,ESTACAO_ENTREGA_ACESSO,CENTRO_ROTEAMENTO,NUVEM_ACESSO,PROVEDOR_ESCOLHA,PROVEDOR_INSTALACAO,PROVEDOR_MENSALIDADE,BBIP,PROTOCOLO_ATENDIMENTO,RETRICAO
207,859091,1203976,REDE 51 - ACESTER,7622693,RT 188 EMPREENDIMENTOS E PARTICIPACOES LTDA,50454665,500Mbps,SPO,2024-12-27,2024-12-27 14:10:45,NaN,0,RPAD,NaN,NaN,NaN,AV,PRESIDENTE JUSCELINO KUBITSCHEK,1455,SL B AN 14,VILA NOVA CONCEICAO,SAO PAULO,SP,SAO PAULO,SP METRO,4543011,BLD - Business Link Direct,500Mbps,1,NaN,Serviço/Velocidade SEV Manual,,"-23,5908400","-46,6821700",NaN,NaN,Viabilidade,Análise,NaN,NaN,NaN,10021539,NAO,NaN,NaN,NaN,NaN,NaN,NAO,NaN,NAO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ativar,PORTFOLIO,500.0,,FOetherNET,FOETHERNET,FO EDD NET,SPO IG,SPO IG,NaN,NaN,NaN,NaN,SAO PAULO|SPO IG,10022113,
222,859147,1204068,REDE 51 - ACESTER,7622822,FERRARA CANDY BRAZIL LTDA,HOLDING SP,500Mbps,SPO,2024-12-27,2024-12-27 16:12:39,NaN,0,RPAD,NaN,NaN,NaN,AV,PRESIDENTE JUSCELINO KUBITSCHEK,1455,SL B AN 14,VILA NOVA CONCEICAO,SAO PAULO,SP,SAO PAULO,SP METRO,4543011,BLD - Business Link Direct,500Mbps,1,NaN,Serviço/Velocidade SEV Manual,,"-23,5908400","-46,6821700",NaN,NaN,Viabilidade,Análise,NaN,NaN,NaN,10021712,NAO,NaN,NaN,NaN,NaN,NaN,NAO,NaN,NAO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ativar,PORTFOLIO,500.0,,,TERCEIROS_ETH,TERCEIRO ETH,SPO IG,SPO IG,WCS / MEGATELECOM / ALGAR TELECOM / UPIX / ASC...,WCS,2500.0,870.0,SAO PAULO|SPO IG,10022129,


In [11]:
restricao[restricao.SEV == 7622822]

,ID,SEV,OBJECTID,ESTADO,SIGLA_DA_LOCALIDADE,NOME,TIPO_DE_IMPACTO,TIPO_RESTRICAO,TIPO_DA_AREA,MEIO_TRANS_IMPACTADO,TECNOLOGIA_IMPACTADA,OBSERVACAO,DATA_ATUALIZACAO,ORIGEM,COD_PRO,POSICAO
43,223,7622822,7662615,SP,SPO,INTERNATIONAL PLAZA II,TOTAL,,CONTRATUAL,ENLACE DE RADIO / REDE METÁLICA / REDE OPTICA,DOCSIS 3.0 / DOCSIS 3.1 / DWDM / FO EDD DC / F...,,12/23/2024 101717 PM,MAPINFO,25,1


In [12]:
for index, value in sevs_tratar[sevs_tratar.TRATADO == 'X'].iterrows():
    aux_restricao = restricao[restricao.SEV == value.SEV]
    if len(aux_restricao) > 0:

        if len(aux_restricao) > 1:
            if 'TOTAL' in aux_restricao.TIPO_DE_IMPACTO.tolist():
                sevs_tratar.at[index,'RESTRICAO'] = aux_restricao.OBSERVACAO.values[0]
                sevs_tratar.at[index,'TRATADO'] = ''

        else:
            if aux_restricao.TIPO_DE_IMPACTO.values[0] == 'TOTAL':
                sevs_tratar.at[index,'RESTRICAO'] = aux_restricao.OBSERVACAO.values[0]
                sevs_tratar.at[index,'TRATADO'] = ''

In [13]:
sevs_tratar[sevs_tratar.TRATADO == 'X'].CENTRO_ROTEAMENTO.unique()

array(['CPE CM', 'BHE ES', 'SPO LP', '', 'SPO IG', 'PMJ SO', 'FNS PO',
       'CTA TF', 'FLA PV', 'IAI C18', 'GNA RD', 'BLM QB', 'MCO MS',
       'VTA JM', 'ILH AP', 'JPA TR', 'RCE AM', 'RJO EN', 'PAE BV',
       'SPO MB', 'IDU 21', 'AMMNS84', 'RJO J3A', 'TSA AN', 'SPO PH',
       'BSA SU', 'MNS C79', 'SDR CPR', 'SLS US', 'CAS BC', 'PAAIU18',
       'RJO AM'], dtype=object)

In [14]:
navegador = webdriver.Chrome(service=service)
navegador.implicitly_wait(5)
navegador.get('http://10.100.1.30/admredes/admredes/RPVB_BLD_Cadastrar_2.asp')
time.sleep(25)

for index, value in sevs_tratar[(sevs_tratar.BBIP != '') & (sevs_tratar.TRATADO == 'X')].iterrows():
    try:
        navegador.get('http://10.100.1.30/admredes/admredes/RPVB_BLD_Cadastrar_2.asp')
        time.sleep(1)
        if (value.BBIP != None) & (value.BBIP != 'BBIP') & ("ID" not in str(value.BBIP)):
            navegador.find_element('name','cliente').send_keys(value.CLIENTE)
            navegador.find_element('name','sev').send_keys(value.SEV)
            municipio, estacao = value.BBIP.split('|')
            municipio = unidecode(municipio).upper()
            id_municipio = str(cidades[cidades.CIDADE == municipio].ID.values[0])
            Select(navegador.find_element('id', 'combo1')).select_by_value(id_municipio)
            Select(navegador.find_element('id', 'combo2')).select_by_visible_text(estacao)
            if 'MPLS' in value.SERVICO or 'BLD' in value.SERVICO:
                Select(navegador.find_element('name', 'servico')).select_by_value('Internet')
            elif 'E-ACCESS' in value.SERVICO or 'EPL' in value.SERVICO:
                Select(navegador.find_element('name', 'servico')).select_by_value('EPL')
            navegador.find_element('name', 'velocidade').send_keys(int(value.VEL))
            navegador.find_element('xpath', '/html/body/div[6]/font/form/div/table[1]/tbody/tr[2]/td/table[5]/tbody/tr/td/input').click()
            time.sleep(4)
            id_bbip = navegador.find_element('xpath','/html/body/div[6]/font/form/div/table[1]/tbody/tr[2]/td/table[1]/tbody/tr/td[1]').text
            status_bbip = navegador.find_element('xpath','/html/body/div[6]/font/form/div/table[1]/tbody/tr[2]/td/table[4]/tbody/tr/td[2]/font/b').text
            sevs_tratar.at[index,'BBIP'] = f'{id_bbip} / {status_bbip}'
    except:
        continue

navegador.close()

In [15]:
for index, value in sevs_tratar[sevs_tratar.BBIP != ''].iterrows():
    if 'Indeferido' in value.BBIP:
        sevs_tratar.at[index,'TRATADO'] = ''

In [16]:
for index, value in sevs_tratar[(sevs_tratar.TRATADO == 'X') & (sevs_tratar.FACILIDADE_ACESSO != 'TERCEIROS_ETH')].iterrows():

    aux_custo = custos_acesso[custos_acesso.FACILIDADE == value.FACILIDADE_ACESSO]

    if value.FACILIDADE_ABORDADO != '':
        sevs_tratar.at[index,'CUSTO_ACESSO_PROPRIO'] = aux_custo.ACESSO_ABORDADO.values[0]
    else:
        if value.VEL <= 202:
            sevs_tratar.at[index,'CUSTO_ACESSO_PROPRIO'] = aux_custo.NOVO_ACESSO_200M.values[0]
        elif value.VEL <= 502:
            sevs_tratar.at[index,'CUSTO_ACESSO_PROPRIO'] = aux_custo.NOVO_ACESSO_500M.values[0]
        elif value.VEL <= 1000:
            sevs_tratar.at[index,'CUSTO_ACESSO_PROPRIO'] = aux_custo.NOVO_ACESSO_1G.values[0]

In [18]:
sevs_tratar[sevs_tratar.FACILIDADE_ABORDADO != '']

,ID_ANALISE,ID_TRIAGEM,CAIXA,SEV,CLIENTE,CLIENTE_FINAL,VELOCIDADE,CNL,DATA_DE_INCLUSAO,DATA_DE_TRIAGEM,DATA_FECHAMENTO_TEIA,DIAS,SOLICITANTE,SOLUCIONADOR,PRIORIZAR,FINALIZADO,TIPO_LOGRADOURO,NOME_DO_LOGRADOURO,NUMERO,COMPLEMENTO,BAIRRO,CIDADE,UF,ESTADO,REGIONAL,CEP,SERVICO,VELOCIDADE_SERV,QTDE_CIRCUITOS,PABX_VIRTUAL,OBS_SOLICITANTE,PONTA_A,LATITUDE,LONGITUDE,CODGEO,INDGEO,FASE,SUBFASE,OBS_PENDENCIA,STATUS_FECHAMENTO,FACILIDADE,PROTOCOLO_GAIA,ABORDADO,FACILIDADE_ABORDADA,PRAZO,PRAZO_MANUAL,JUSTIFICATIVA_PRAZO_MANUAL,DISPONIBILIDADE,RESERVA_BACKBONE_IP,PROTOCOLO_IP,RESERVA_BACKBONE_TELEFONIA,PROTOCOLO_TELEFONIA,ESTACAO_DE_ENTREGA,EMPRESA_FORNECEDORA,CUSTO_TOTAL,MENSALIDADE_TERCEIRO,TAXA_DE_INSTALACAO_TERCEIRO,CUSTO_ACESSO,CUSTO_ACESSO_EQUIPAMENTO,CUSTO_PLANTA_EQUIPAMENTO,CUSTO_PLANTA_REDE_EXTERNA,CUSTO_EXCLUSIVO_ACESSO,RESPOSTA_SEV,ACAO,PROJETO,VEL,TRATADO,FACILIDADE_ABORDADO,FACILIDADE_ACESSO,NUVEM_ACESSO,ESTACAO_ENTREGA_ACESSO,TECNOLOGIA_ACESSO,CENTRO_ROTEAMENTO,PROVEDOR_ESCOLHA,PROVEDOR_INSTALACAO,PROVEDOR_MENSALIDADE,BBIP,PROTOCOLO_ATENDIMENTO,RESTRICAO,CUSTO_ACESSO_PROPRIO


In [21]:
sevs_tratar[sevs_tratar.SEV == 7632326]

,ID_ANALISE,ID_TRIAGEM,CAIXA,SEV,CLIENTE,CLIENTE_FINAL,VELOCIDADE,CNL,DATA_DE_INCLUSAO,DATA_DE_TRIAGEM,DATA_FECHAMENTO_TEIA,DIAS,SOLICITANTE,SOLUCIONADOR,PRIORIZAR,FINALIZADO,TIPO_LOGRADOURO,NOME_DO_LOGRADOURO,NUMERO,COMPLEMENTO,BAIRRO,CIDADE,UF,ESTADO,REGIONAL,CEP,SERVICO,VELOCIDADE_SERV,QTDE_CIRCUITOS,PABX_VIRTUAL,OBS_SOLICITANTE,PONTA_A,LATITUDE,LONGITUDE,CODGEO,INDGEO,FASE,SUBFASE,OBS_PENDENCIA,STATUS_FECHAMENTO,FACILIDADE,PROTOCOLO_GAIA,ABORDADO,FACILIDADE_ABORDADA,PRAZO,PRAZO_MANUAL,JUSTIFICATIVA_PRAZO_MANUAL,DISPONIBILIDADE,RESERVA_BACKBONE_IP,PROTOCOLO_IP,RESERVA_BACKBONE_TELEFONIA,PROTOCOLO_TELEFONIA,ESTACAO_DE_ENTREGA,EMPRESA_FORNECEDORA,CUSTO_TOTAL,MENSALIDADE_TERCEIRO,TAXA_DE_INSTALACAO_TERCEIRO,CUSTO_ACESSO,CUSTO_ACESSO_EQUIPAMENTO,CUSTO_PLANTA_EQUIPAMENTO,CUSTO_PLANTA_REDE_EXTERNA,CUSTO_EXCLUSIVO_ACESSO,RESPOSTA_SEV,ACAO,PROJETO,VEL,TRATADO,FACILIDADE_ABORDADO,FACILIDADE_ACESSO,TECNOLOGIA_ACESSO,ESTACAO_ENTREGA_ACESSO,CENTRO_ROTEAMENTO,NUVEM_ACESSO,PROVEDOR_ESCOLHA,PROVEDOR_INSTALACAO,PROVEDOR_MENSALIDADE,BBIP,PROTOCOLO_ATENDIMENTO,RESTRICAO,CUSTO_ACESSO_PROPRIO
122,863991,1210939,NaN,7632326,AGROPECUARIA RONCADOR,AGROPECUARIA RONCADOR,25Mbps,QRNC,2025-01-08,2025-01-08 19:12:00,NaN,1,GDAIANA,NaN,NaN,NaN,ROD,MT 109,109,KM 70,ZONA RURAL,QUERENCIA,MT,MATO GROSSO,CONO,78643000,ISM - IPSAT,25Mbps,1,NaN,Nome Nuvem: Sat D2 - BEAM 6,,"-12,1089227","-52,198163",NaN,NaN,Viabilidade,Análise,NaN,NaN,NaN,10037148,NAO,NaN,NaN,NaN,NaN,NaN,NAO,NaN,NAO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ativar,PORTFOLIO,25.0,X,,SATELITE_BANDA_KA,BANDA KA,RJO AM,,,NaN,NaN,NaN,,10038051,NaN,4904.0


In [19]:
for index, value in sevs_tratar[sevs_tratar.TRATADO == 'X'].iterrows():
    if value.FACILIDADE_ACESSO != 'TERCEIROS_ETH':
        aux_facilidade = tecnologia_facilidade[tecnologia_facilidade.FACILIDADE == value.FACILIDADE_ACESSO]
        if len(aux_facilidade) > 0:
            sevs_tratar.at[index,'FACILIDADE_FECHAMENTO'] = aux_facilidade.FACILIDADE_FECHAMENTO.values[0]
            sevs_tratar.at[index,'ID_FACILIDADE'] = aux_facilidade.ID.values[0]
            sevs_tratar.at[index,'EMPRESA_FACILIDADE'] = aux_facilidade.EMPRESA.values[0]
            sevs_tratar.at[index,'ID_EMPRESA_FACILIDADE'] = aux_facilidade.ID_EMPRESA.values[0]
    else:
        aux_facilidade = tecnologia_facilidade[tecnologia_facilidade.FACILIDADE == value.FACILIDADE_ACESSO]
        aux_empresas = id_provedores[id_provedores.PROVEDOR_TEIA.str.contains(value.PROVEDOR_ESCOLHA)]
        if len(aux_empresas) > 1:
            aux_empresas = id_provedores[id_provedores.PROVEDOR_TEIA.str.contains(f'{value.PROVEDOR_ESCOLHA} - {value.UF}')]
            if len(aux_empresas) > 0:
                sevs_tratar.at[index,'FACILIDADE_FECHAMENTO'] = aux_facilidade.FACILIDADE_FECHAMENTO.values[0]
                sevs_tratar.at[index,'ID_FACILIDADE'] = aux_facilidade.ID.values[0]
                sevs_tratar.at[index,'EMPRESA_FACILIDADE'] = aux_empresas.PROVEDOR_TEIA.values[0]
                sevs_tratar.at[index,'ID_EMPRESA_FACILIDADE'] = aux_empresas.ID.values[0]
            else:
                aux_empresas = id_provedores[id_provedores.PROVEDOR_TEIA.str.contains(value.PROVEDOR_ESCOLHA)]
                sevs_tratar.at[index,'FACILIDADE_FECHAMENTO'] = aux_facilidade.FACILIDADE_FECHAMENTO.values[0]
                sevs_tratar.at[index,'ID_FACILIDADE'] = aux_facilidade.ID.values[0]
                sevs_tratar.at[index,'EMPRESA_FACILIDADE'] = aux_empresas.PROVEDOR_TEIA.values[0]
                sevs_tratar.at[index,'ID_EMPRESA_FACILIDADE'] = aux_empresas.ID.values[0]

        elif len(aux_empresas) == 1:
            sevs_tratar.at[index,'FACILIDADE_FECHAMENTO'] = aux_facilidade.FACILIDADE_FECHAMENTO.values[0]
            sevs_tratar.at[index,'ID_FACILIDADE'] = aux_facilidade.ID.values[0]
            sevs_tratar.at[index,'EMPRESA_FACILIDADE'] = aux_empresas.PROVEDOR_TEIA.values[0]
            sevs_tratar.at[index,'ID_EMPRESA_FACILIDADE'] = aux_empresas.ID.values[0]


sevs_tratar = sevs_tratar.fillna('')

In [20]:
for i,v in sevs_tratar.iterrows():
    if v.EMPRESA_FACILIDADE == '':
        sevs_tratar.at[i,'TRATADO'] = ''

    if v.CENTRO_ROTEAMENTO == '':
        sevs_tratar.at[i,'CENTRO_ROTEAMENTO'] = estacoes_uf[estacoes_uf.UF == v.UF].ESTACAO.values[0]

In [21]:
for index, value in sevs_tratar[sevs_tratar.TRATADO == 'X'].iterrows():
    aux_estacao_teia = estacoes_newteia[estacoes_newteia.old_id == value.CENTRO_ROTEAMENTO]
    if len(aux_estacao_teia) > 0:
        sevs_tratar.at[index,'CENTRO_ROTEAMENTO'] = aux_estacao_teia.estacao.values[0]

    else:
        aux_estacao_teia = estacoes_newteia[estacoes_newteia.estacao == value.CENTRO_ROTEAMENTO]
        if len(aux_estacao_teia) > 0:
            sevs_tratar.at[index,'CENTRO_ROTEAMENTO'] = aux_estacao_teia.estacao.values[0]
        else:
            
            sevs_tratar.at[index,'CENTRO_ROTEAMENTO'] = estacoes_uf[estacoes_uf.UF == value.UF].ESTACAO.values[0]

In [ ]:
for index, value in sevs_tratar[sevs_tratar.TRATADO == 'X'].iterrows():
    if len(fechamento_teia) == 0:
        fechamento_teia.at[0,'sequencial'] = value.SEV
        fechamento_teia.loc[len(fechamento_teia) - 1,['latitude','longitude','uf','cnl','facilidade','id_facilidade',
        'provedor','id_provedor','entrega','id_da_sev','prazo','codigo_spe','sinalizacao_sip',
        'protocolo_gaia','status']] = [value.LATITUDE, value.LONGITUDE, value.UF, value.CNL, value.FACILIDADE_FECHAMENTO, str(value.ID_FACILIDADE).split('.')[0],
        value.EMPRESA_FACILIDADE,str(value.ID_EMPRESA_FACILIDADE).split('.')[0], value.CENTRO_ROTEAMENTO, value.ID_ANALISE, '', '','',value.PROTOCOLO_ATENDIMENTO,'1']
        if value.FACILIDADE_ABORDADO != '':
            fechamento_teia.at[len(fechamento_teia) - 1,'abordado'] = 'SIM'
        else:
            fechamento_teia.at[len(fechamento_teia) - 1,'abordado'] = 'NAO'
        if value.FACILIDADE_ACESSO != 'TERCEIROS_ETH':
            fechamento_teia.at[len(fechamento_teia) - 1,'custo_de_acesso_proprio'] = str(value.CUSTO_ACESSO_PROPRIO).replace('.',',')
            fechamento_teia.at[len(fechamento_teia) - 1,'obs'] = f'FECHAMENTO NFV/ ESTACAO ENTREGA {value.ESTACAO_ENTREGA_ACESSO}/{value.NUVEM_ACESSO}'

        else:
            fechamento_teia.at[len(fechamento_teia) - 1,'instalacao_terceiros'] = str(value.PROVEDOR_INSTALACAO).replace('.',',')
            fechamento_teia.at[len(fechamento_teia) - 1,'mensalidade_terceiros'] = str(value.PROVEDOR_MENSALIDADE).replace('.',',')
            fechamento_teia.at[len(fechamento_teia) - 1,'tipo_terceiros'] = 3
            fechamento_teia.at[len(fechamento_teia) - 1,'justificativa'] = 'FORA DE REDE'
            fechamento_teia.at[len(fechamento_teia) - 1,'ID Justificativa'] = 1
            fechamento_teia.at[len(fechamento_teia) - 1,'obs'] = f'FECHAMENTO NFV FASE 1/ ESTACAO ENTREGA {value.ESTACAO_ENTREGA_ACESSO}'

        if value.BBIP != '':
            fechamento_teia.at[len(fechamento_teia) - 1,'bbip'] = value.BBIP.split(' / ')[0].split(': ')[-1]
    
    else:
        fechamento_teia.at[len(fechamento_teia),'sequencial'] = value.SEV
        fechamento_teia.loc[len(fechamento_teia) - 1,['latitude','longitude','uf','cnl','facilidade','id_facilidade',
        'provedor','id_provedor','entrega','id_da_sev','prazo','codigo_spe','sinalizacao_sip',
        'protocolo_gaia','status']] = [value.LATITUDE, value.LONGITUDE, value.UF, value.CNL, value.FACILIDADE_FECHAMENTO, str(value.ID_FACILIDADE).split('.')[0],
        value.EMPRESA_FACILIDADE,str(value.ID_EMPRESA_FACILIDADE).split('.')[0], value.CENTRO_ROTEAMENTO, value.ID_ANALISE, '', '','',value.PROTOCOLO_ATENDIMENTO,'1']
        if value.FACILIDADE_ABORDADO != '':
            fechamento_teia.at[len(fechamento_teia) - 1,'abordado'] = 'SIM'
        else:
            fechamento_teia.at[len(fechamento_teia) - 1,'abordado'] = 'NAO'
        if value.FACILIDADE_ACESSO != 'TERCEIROS_ETH':
            fechamento_teia.at[len(fechamento_teia) - 1,'custo_de_acesso_proprio'] = str(value.CUSTO_ACESSO_PROPRIO).replace('.',',')
            fechamento_teia.at[len(fechamento_teia) - 1,'obs'] = f'FECHAMENTO NFV FASE 1/ ESTACAO ENTREGA {value.ESTACAO_ENTREGA_ACESSO}/{value.NUVEM_ACESSO}'

        else:
            fechamento_teia.at[len(fechamento_teia) - 1,'instalacao_terceiros'] = str(value.PROVEDOR_INSTALACAO).replace('.',',')
            fechamento_teia.at[len(fechamento_teia) - 1,'mensalidade_terceiros'] = str(value.PROVEDOR_MENSALIDADE).replace('.',',')
            fechamento_teia.at[len(fechamento_teia) - 1,'tipo_terceiros'] = 3
            fechamento_teia.at[len(fechamento_teia) - 1,'justificativa'] = 'FORA DE REDE'
            fechamento_teia.at[len(fechamento_teia) - 1,'ID Justificativa'] = 1
            fechamento_teia.at[len(fechamento_teia) - 1,'obs'] = f'FECHAMENTO NFV FASE 1/ ESTACAO ENTREGA {value.ESTACAO_ENTREGA_ACESSO}'

        if value.BBIP != '':
            fechamento_teia.at[len(fechamento_teia) - 1,'bb_ip'] = value.BBIP.split(' / ')[0].split(': ')[-1]

        if value.FACILIDADE_ACESSO == 'HFC_BSOD':
            if 'HP GED' in value.NUVEM_ACESSO:
                fechamento_teia.at[len(fechamento_teia) - 1,'hp_bsod'] = value.NUVEM_ACESSO.split('HP GED ')[-1]

fechamento_teia = fechamento_teia.fillna('')

In [23]:
fechamento_teia.to_csv('fechamento_lote_nfv.csv',sep=';',index=False)
sevs_tratar[sevs_tratar.TRATADO == ''].to_excel('fechamento_nfv_lote_manual.xlsx',index=False)

In [24]:
# for index,value in sevs_tratar[sevs_tratar.SERVICO.isin(['LAN - LAN EPL','LAN - LAN EPL MEF'])].iterrows():
#     if value.PONTA_A == '':
#         aux_sev = sevs_tratar[sevs_tratar.PONTA_A == value.SEV]
#         if len(aux_sev) > 0:
#             if (value.FACILIDADE_ACESSO != aux_sev.FACILIDADE_ACESSO.values[0]) & (value.FACILIDADE_ACESSO != '') & (aux_sev.FACILIDADE_ACESSO.values[0] != ''):
#                 prioridade_ponta_a = facilidades[facilidades.FACILIDADE == value.FACILIDADE_ACESSO].PRIORIDADE.values[0]
#                 prioridade_ponta_b = facilidades[facilidades.FACILIDADE == aux_sev.FACILIDADE_ACESSO.values[0]].PRIORIDADE.values[0]

#                 if (prioridade_ponta_a > prioridade_ponta_b) & ((prioridade_ponta_a != 10) & (prioridade_ponta_b != 10)):
#                     if (prioridade_ponta_a in [8,9]):
#                         if prioridade_ponta_b in [8,9]:
#                             continue

#                     index_pt_b = aux_sev.index[0]
#                     sev_pt_b = aux_sev.SEV.values[0]
#                     sevs_tratar.at[index_pt_b,'TRATADO'] = ''
#                     for i,v in facilidades[facilidades.PRIORIDADE > prioridade_ponta_b].iterrows():
#                         for tec in v.TECNOLOGIA.split('/'):
#                             if sevs_tratar.at[index_pt_b,'TRATADO'] != 'X':
#                                 aux_resumosoe = resumosoe[resumosoe.SEV == sev_pt_b]
#                                 if len(aux_resumosoe) > 0:
#                                     for nome_nuvem in aux_resumosoe[f'{v.FACILIDADE}_INFORMACAO'].values[0].split(' / '):
#                                         aux_nuvem = nuvens[(nuvens.SEV == sev_pt_b) & (nuvens.NOME_NUVEM == nome_nuvem) & (nuvens.TECNOLOGIA == tec)]
#                                         if len(aux_nuvem) > 0:
#                                             if (aux_sev.SERVICO.values[0] == 'LAN - LAN EPL MEF') & (aux_sev.VEL.values[0] <= aux_nuvem.CAPACITY_NUVEM.values[0]):
                                                
#                                                 if 'EPL MEF - NOK' not in aux_nuvem.OBSERVACAO.values[0]:
                                                    
#                                                     sevs_tratar.at[index_pt_b,'FACILIDADE_ACESSO'] = v.FACILIDADE
#                                                     if tec in v.CONSULTA_FUNIL.split('/'):
#                                                         sevs_tratar.at[index_pt_b,'NUVEM_ACESSO'] = f'NUVEM {aux_nuvem.NOME_NUVEM.values[0]} SITE {aux_nuvem.SIGLA_ESTACAO_CLARO.values[0]}'
#                                                     else:
#                                                         sevs_tratar.at[index_pt_b,'NUVEM_ACESSO'] = aux_nuvem.NOME_NUVEM.values[0]
#                                                     sevs_tratar.at[index_pt_b,'ESTACAO_ENTREGA_ACESSO'] = aux_nuvem.ESTACAO_ENTREGA.values[0]
#                                                     sevs_tratar.at[index_pt_b,'TECNOLOGIA_ACESSO'] = tec
#                                                     sevs_tratar.at[index_pt_b,'TRATADO'] = 'X'
#                                                     break

#                                             if (aux_sev.SERVICO.values[0] != 'LAN - LAN EPL MEF') & (aux_sev.VEL.values[0] <= aux_nuvem.CAPACITY_NUVEM.values[0]):
#                                                 sevs_tratar.at[index_pt_b,'FACILIDADE_ACESSO'] = v.FACILIDADE
#                                                 if tec in v.CONSULTA_FUNIL.split('/'):
#                                                     sevs_tratar.at[index_pt_b,'NUVEM_ACESSO'] = f'NUVEM {aux_nuvem.NOME_NUVEM.values[0]} SITE {aux_nuvem.SIGLA_ESTACAO_CLARO.values[0]}'
#                                                 else:
#                                                     sevs_tratar.at[index_pt_b,'NUVEM_ACESSO'] = aux_nuvem.NOME_NUVEM.values[0]
#                                                 sevs_tratar.at[index_pt_b,'ESTACAO_ENTREGA_ACESSO'] = aux_nuvem.ESTACAO_ENTREGA.values[0]
#                                                 sevs_tratar.at[index_pt_b,'TECNOLOGIA_ACESSO'] = tec
#                                                 sevs_tratar.at[index_pt_b,'TRATADO'] = 'X'
#                                                 break
#                 elif (prioridade_ponta_a < prioridade_ponta_b) & ((prioridade_ponta_a != 10) & (prioridade_ponta_b != 10)):
#                     if (prioridade_ponta_a in [8,9]):
#                         if prioridade_ponta_b in [8,9]:
#                             continue

#                     for i,v in facilidades[facilidades.PRIORIDADE > prioridade_ponta_b].iterrows():
#                         for tec in v.TECNOLOGIA.split('/'):
#                             if sevs_tratar.at[index,'TRATADO'] != 'X':
#                                 aux_resumosoe = resumosoe[resumosoe.SEV == sev_pt_b]
#                                 if len(aux_resumosoe) > 0:
#                                     for nome_nuvem in aux_resumosoe[f'{v.FACILIDADE}_INFORMACAO'].values[0].split(' / '):
#                                         aux_nuvem = nuvens[(nuvens.SEV == value.SEV) & (nuvens.NOME_NUVEM == nome_nuvem) & (nuvens.TECNOLOGIA == tec)]
#                                         if len(aux_nuvem) > 0:
#                                             if (value.SERVICO == 'LAN - LAN EPL MEF') & (value.VEL <= aux_nuvem.CAPACITY_NUVEM.values[0]):
#                                                 # atualizar logica para mef com edd fixa ou dc e pv apenas
#                                                 # atualizar logica para epl ser viavel pontas gpon foethernet edd fixa ou edd movel, dc ou pv
#                                                 if 'EPL MEF - NOK' not in aux_nuvem.OBSERVACAO.values[0]:
#                                                     sevs_tratar.at[index,'FACILIDADE_ACESSO'] = v.FACILIDADE
#                                                     if tec in v.CONSULTA_FUNIL.split('/'):
#                                                         sevs_tratar.at[index,'NUVEM_ACESSO'] = f'NUVEM {aux_nuvem.NOME_NUVEM.values[0]} SITE {aux_nuvem.SIGLA_ESTACAO_CLARO.values[0]}'
#                                                     else:
#                                                         sevs_tratar.at[index,'NUVEM_ACESSO'] = aux_nuvem.NOME_NUVEM.values[0]
#                                                     sevs_tratar.at[index,'ESTACAO_ENTREGA_ACESSO'] = aux_nuvem.ESTACAO_ENTREGA.values[0]
#                                                     sevs_tratar.at[index,'TECNOLOGIA_ACESSO'] = tec
#                                                     sevs_tratar.at[index,'TRATADO'] = 'X'
#                                                     break

#                                             if (value.SERVICO != 'LAN - LAN EPL MEF') & (value.VEL <= aux_nuvem.CAPACITY_NUVEM.values[0]):
#                                                 sevs_tratar.at[index,'FACILIDADE_ACESSO'] = v.FACILIDADE
#                                                 if tec in v.CONSULTA_FUNIL.split('/'):
#                                                     sevs_tratar.at[index,'NUVEM_ACESSO'] = f'NUVEM {aux_nuvem.NOME_NUVEM.values[0]} SITE {aux_nuvem.SIGLA_ESTACAO_CLARO.values[0]}'
#                                                 else:
#                                                     sevs_tratar.at[index,'NUVEM_ACESSO'] = aux_nuvem.NOME_NUVEM.values[0]
#                                                 sevs_tratar.at[index,'ESTACAO_ENTREGA_ACESSO'] = aux_nuvem.ESTACAO_ENTREGA.values[0]
#                                                 sevs_tratar.at[index,'TECNOLOGIA_ACESSO'] = tec
#                                                 sevs_tratar.at[index,'TRATADO'] = 'X'
#                                                 break
#                 else:
                    
#                     sevs_tratar.at[index,'FACILIDADE_ACESSO'] = ''
#                     sevs_tratar.at[index,'NUVEM_ACESSO'] = ''
#                     sevs_tratar.at[index,'ESTACAO_ENTREGA_ACESSO'] = ''
#                     sevs_tratar.at[index,'TECNOLOGIA_ACESSO'] = ''
#                     sevs_tratar.at[index,'TRATADO'] = ''

                    
#                     index_pt_b = aux_sev.index[0]

#                     sevs_tratar.at[index_pt_b,'FACILIDADE_ACESSO'] = ''
#                     sevs_tratar.at[index_pt_b,'NUVEM_ACESSO'] = ''
#                     sevs_tratar.at[index_pt_b,'ESTACAO_ENTREGA_ACESSO'] = ''
#                     sevs_tratar.at[index_pt_b,'TECNOLOGIA_ACESSO'] = ''
#                     sevs_tratar.at[index_pt_b,'TRATADO'] = ''
                    
#             elif (value.FACILIDADE_ACESSO == '') | (aux_sev.FACILIDADE_ACESSO.values[0] == ''):
                
#                 sevs_tratar.at[index,'FACILIDADE_ACESSO'] = ''
#                 sevs_tratar.at[index,'NUVEM_ACESSO'] = ''
#                 sevs_tratar.at[index,'ESTACAO_ENTREGA_ACESSO'] = ''
#                 sevs_tratar.at[index,'TECNOLOGIA_ACESSO'] = ''
#                 sevs_tratar.at[index,'TRATADO'] = ''
                
#                 index_pt_b = aux_sev.index[0]

#                 sevs_tratar.at[index_pt_b,'FACILIDADE_ACESSO'] = ''
#                 sevs_tratar.at[index_pt_b,'NUVEM_ACESSO'] = ''
#                 sevs_tratar.at[index_pt_b,'ESTACAO_ENTREGA_ACESSO'] = ''
#                 sevs_tratar.at[index_pt_b,'TECNOLOGIA_ACESSO'] = ''
#                 sevs_tratar.at[index_pt_b,'TRATADO'] = ''
#         else:
#             sevs_tratar.at[index,'FACILIDADE_ACESSO'] = ''
#             sevs_tratar.at[index,'NUVEM_ACESSO'] = ''
#             sevs_tratar.at[index,'ESTACAO_ENTREGA_ACESSO'] = ''
#             sevs_tratar.at[index,'TECNOLOGIA_ACESSO'] = ''
#             sevs_tratar.at[index,'TRATADO'] = ''


In [169]:
sevs_tratar[(sevs_tratar.VEL <= 102) & (sevs_tratar.FACILIDADE_VIABILIDADE.isin(['HFC_BSOD', 'FO_XGSPON_RESID_ETH', 'FO_GPON_RESID_ETH',
       'FO_GPON_ETH', 'FOETHERNET', 'FO_EDD_ETH', 'FO_EDD_DIRETO_CLARO',
       'FO_EDD_CLARO_PRE_VIAVEL', 'FO_SDH', '4G', 'TERCEIROS_ETH' ]))]

,ID_ANALISE,ID_TRIAGEM,CAIXA,SEV,CLIENTE,CLIENTE_FINAL,VELOCIDADE,CNL,DATA_DE_INCLUSAO,DATA_DE_TRIAGEM,DATA_FECHAMENTO_TEIA,DIAS,SOLICITANTE,SOLUCIONADOR,PRIORIZAR,FINALIZADO,TIPO_LOGRADOURO,NOME_DO_LOGRADOURO,NUMERO,COMPLEMENTO,BAIRRO,CIDADE,UF,ESTADO,REGIONAL,CEP,SERVICO,VELOCIDADE_SERV,QTDE_CIRCUITOS,PABX_VIRTUAL,OBS_SOLICITANTE,PONTA_A,LATITUDE,LONGITUDE,CODGEO,INDGEO,FASE,SUBFASE,OBS_PENDENCIA,STATUS_FECHAMENTO,FACILIDADE,PROTOCOLO_GAIA,ABORDADO,FACILIDADE_ABORDADA,PRAZO,PRAZO_MANUAL,JUSTIFICATIVA_PRAZO_MANUAL,DISPONIBILIDADE,RESERVA_BACKBONE_IP,PROTOCOLO_IP,RESERVA_BACKBONE_TELEFONIA,PROTOCOLO_TELEFONIA,ESTACAO_DE_ENTREGA,EMPRESA_FORNECEDORA,CUSTO_TOTAL,MENSALIDADE_TERCEIRO,TAXA_DE_INSTALACAO_TERCEIRO,CUSTO_ACESSO,CUSTO_ACESSO_EQUIPAMENTO,CUSTO_PLANTA_EQUIPAMENTO,CUSTO_PLANTA_REDE_EXTERNA,CUSTO_EXCLUSIVO_ACESSO,RESPOSTA_SEV,ACAO,PROJETO,VEL,TRATADO,FACILIDADE_VIABILIDADE,FACILIDADE_EMPRESA,FACILIDADE_ESTACAO,FACILIDADE_NUVEM,CLIENTE_ABORDADO,FACILIDADE_TECNOLOGIA,FACILIDADE_SITE
3,782115,1098635,REDE 66 - MS,7450845,REDE D OR SAO LUIZ S A,REDE DOR,50Mbps,NAAD,2024-09-06,2024-09-06 11:12:29,NaN,1,ASILVAP,NaN,NaN,NaN,R,ANTONIO CARLOS BARBOSA,1495,NaN,CENTRO,NOVA ALVORADA DO SUL,MS,MATO GROSSO DO SUL,CONO,79140000,BLD - Business Link Direct,50Mbps,1,NaN,Facilidade Terceiros,NaN,-21.448108,-54.370626,NaN,NaN,Viabilidade,Análise,NaN,NaN,NaN,9692238,NAO,NaN,NaN,NaN,NaN,NaN,NAO,NaN,NAO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ativar,PORTFOLIO,50.0,X,TERCEIROS_ETH,IG NETWORKS / SITELBRA,CPE CM,NUVEM: NOVA ALVORADA DO SUL - PROPRIETÁRIO NUV...,N,NaN,NaN
4,782116,1098636,REDE 66 - MS,7450848,REDE D OR SAO LUIZ S A,REDE DOR,50Mbps,KCA,2024-09-06,2024-09-06 11:12:29,NaN,1,ASILVAP,NaN,NaN,NaN,R,1,100,NaN,CENTRO,COSTA RICA,MS,MATO GROSSO DO SUL,CONO,79550000,BLD - Business Link Direct,50Mbps,1,NaN,Facilidade Terceiros,NaN,-18.528171,-53.149472,NaN,NaN,Viabilidade,Análise,NaN,NaN,NaN,9692242,NAO,NaN,NaN,NaN,NaN,NaN,NAO,NaN,NAO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ativar,PORTFOLIO,50.0,X,TERCEIROS_ETH,G8 TELECOM / IG NETWORKS / SITELBRA,CPE CM,NUVEM: COSTA RICA-MS 01 - PROPRIETÁRIO NUVEM: ...,N,NaN,NaN
5,782117,1098637,REDE 42 - BA,7450850,DB3,DB3,100Mbps,CIS,2024-09-06,2024-09-06 11:12:29,NaN,1,CAALEX,NaN,NaN,NaN,ROD,BR 324,0,NaN,DISTRITO INDUSTRIAL,CANDEIAS,BA,BAHIA,BA/SE,43813400,BLD - Business Link Direct,100Mbps,1,NaN,Facilidade Terceiros,NaN,-12.629143,-38.466893,NaN,NaN,Viabilidade,Análise,NaN,NaN,NaN,9692245,NAO,NaN,NaN,NaN,NaN,NaN,NAO,NaN,NAO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ativar,PORTFOLIO,100.0,X,TERCEIROS_ETH,IG NETWORKS,SDR RC,NUVEM: CANDEIAS - PROPRIETÁRIO NUVEM: IG NETWORKS,N,NaN,NaN
10,782129,1098649,SERVICO - REDE 41 - ACESTER-RJ,7450868,UNILIDER DISTRIBUIDORA S A,UNILIDER,50Mbps,SFD,2024-09-06,2024-09-06 11:12:31,NaN,1,FJNASCI,NaN,NaN,NaN,AV,EMYGDIO MAIA SANTOS,296,NaN,VILA DOS COROADOS,SAO FIDELIS,RJ,RIO DE JANEIRO,RJ/ES,28400000,BLD - Business Link Direct,50Mbps,1,NaN,Facilidade Terceiros,NaN,-21.630163,-41.764702,NaN,NaN,Viabilidade,Análise,NaN,NaN,NaN,9692269,NAO,NaN,NaN,NaN,NaN,NaN,NAO,NaN,NAO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ativar,PORTFOLIO,50.0,X,TERCEIROS_ETH,K2 TELECOM / IG NETWORKS,RJO AM,NUVEM: ABRANGENCIA SAO FIDELIS - PROPRIETÁRIO ...,N,NaN,NaN
15,782136,1098656,REDE 51 - ACESTER,7450887,ALEXANDRIA LAND OF IDEAS - COMUNICACAO CONSULT...,PALOMA,100Mbps,SPO,2024-09-06,2024-09-06 11:12:32,NaN,1,F111887,NaN,NaN,NaN,R,FORMOSA,367,NaN,CENTRO,SAO PAULO,SP,SAO PAULO,SP METRO,1049911,BLD - Business Link Direct,100Mbps,1,NaN,Facilidade Terceiros,NaN,-23.545115,-46.636891,NaN,NaN,Viabilidade,Análise,NaN,NaN,NaN,9692292,NAO,NaN,NaN,NaN,NaN,NaN,NAO,NaN,NAO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ativar,PORTFOLIO,100.0,X,FO_SDH,EMBRATEL,SPO IG,,S,OUTRAS,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [3]:
sevs_tratar[sevs_tratar.SEV == 7453939]

,ID_ANALISE,ID_TRIAGEM,CAIXA,SEV,CLIENTE,CLIENTE_FINAL,VELOCIDADE,CNL,DATA_DE_INCLUSAO,DATA_DE_TRIAGEM,DATA_FECHAMENTO_TEIA,DIAS,SOLICITANTE,SOLUCIONADOR,PRIORIZAR,FINALIZADO,TIPO_LOGRADOURO,NOME_DO_LOGRADOURO,NUMERO,COMPLEMENTO,BAIRRO,CIDADE,UF,ESTADO,REGIONAL,CEP,SERVICO,VELOCIDADE_SERV,QTDE_CIRCUITOS,PABX_VIRTUAL,OBS_SOLICITANTE,PONTA_A,LATITUDE,LONGITUDE,CODGEO,INDGEO,FASE,SUBFASE,OBS_PENDENCIA,STATUS_FECHAMENTO,FACILIDADE,PROTOCOLO_GAIA,ABORDADO,FACILIDADE_ABORDADA,PRAZO,PRAZO_MANUAL,JUSTIFICATIVA_PRAZO_MANUAL,DISPONIBILIDADE,RESERVA_BACKBONE_IP,PROTOCOLO_IP,RESERVA_BACKBONE_TELEFONIA,PROTOCOLO_TELEFONIA,ESTACAO_DE_ENTREGA,EMPRESA_FORNECEDORA,CUSTO_TOTAL,MENSALIDADE_TERCEIRO,TAXA_DE_INSTALACAO_TERCEIRO,CUSTO_ACESSO,CUSTO_ACESSO_EQUIPAMENTO,CUSTO_PLANTA_EQUIPAMENTO,CUSTO_PLANTA_REDE_EXTERNA,CUSTO_EXCLUSIVO_ACESSO,RESPOSTA_SEV,ACAO,PROJETO,VEL,TRATADO,FACILIDADE_VIABILIDADE,FACILIDADE_EMPRESA,FACILIDADE_ESTACAO,FACILIDADE_NUVEM,CLIENTE_ABORDADO,FACILIDADE_TECNOLOGIA,FACILIDADE_SITE
230,782865,1099712,REDE 67 - RS,7453939,TELMEX DO BRASIL S A,O BOTICARIO,20Mbps,SPS,2024-09-07,2024-09-07 19:10:37,NaN,0,Z210793,NaN,NaN,NaN,ROD,BR 116,958,NaN,TRES PORTOS,SAPUCAIA DO SUL,RS,RIO GRANDE DO SUL,SUL,93212220,BLD - Business Link Direct,20Mbps,1,NaN,"Facilidade FO GPON ETH SEV Manual, nuvem AVALI...",NaN,-29.806518,-51.16474,NaN,NaN,Viabilidade,Análise,NaN,NaN,NaN,9695628,NAO,NaN,NaN,NaN,NaN,NaN,NAO,NaN,NAO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ativar,PORTFOLIO,20.0,X,FO_GPON_ETH,EMBRATEL,SLE JC,,S,,NaN


In [140]:
nuvens[nuvens.SEV == 7453939]

,SEV,OBJECTID,ESTACAO_ENTREGA,UF,SIGLA_LOC,NOME_NUVEM,REDE,TECNOLOGIA,MEIO_TRANSMISSAO,SITUACAO,ALT_NUVEM,PADRAO_PON,PROPRIETARIO,OBSERVACAO,ORIGEM,DATA_ATUALIZACAO,DATA_PREVISAO,VEL_MAX_VIABILIDADE,VEL_MAX_SEV_AUTOMATICA,MOTIVO,OPERADORA,NUMERO_OPERACIONAL,SIGLA_ESTACAO_CLARO,CODIGO_DESCARGA_CSL,SIGLA_ESTACAO_RESID,TIPO_INFRA,TIPO,STATUS,DONO,ROTEADORES_QTD,TX,FABRICANTE_OLT,ABRANGENCIA,CONCENTRADOR_OLT
560,7453939,7399561,CAN DM,RS,SPS,ANEL_CANMBO-02,CAN MBO,SDH,REDE OPTICA,AVALIAR CAPACIDADE,,,CLARO BRASIL,DM4001_CANMBO-01,MAPINFO,9/2/2024 101330 PM,,,,CAPACIDADE_BB,,22450-27946,,,,,,,,,,,,
561,7453939,7399557,CAN MBO,RS,SPS,ANEL_CANMBO-01,CAN MBO,SDH,REDE OPTICA,AVALIAR CAPACIDADE,,,CLARO BRASIL,DM4001_CANMBO-01,MAPINFO,9/2/2024 101330 PM,,,,CAPACIDADE_BB,,22450-27946,,,,,,,,,,,,
562,7453939,7399556,SLE JC,RS,SPS,SLEJC_ANEL-01,SLE JC,FO EDD FIXA / GPON FIXA,REDE OPTICA,CONCLUIDA,,,CLARO BRASIL,,MAPINFO,9/2/2024 101330 PM,,,,,,,,,,,,,,,,,,C320-SLEJC-001/ZTE
563,7453939,7399555,CAN MBO,RS,SPS,ANEL_CANMBO,CAN MBO,FO EDD FIXA,REDE OPTICA,CONCLUIDA,,,CLARO BRASIL,DM4001_CANMBO-01,MAPINFO,9/2/2024 101329 PM,,,,,,,,,,,,,,,,,,
564,7453939,7399554,CAN CL0,RS,SPS,ANEL_CANMBO-04,CAN MBO,FO EDD FIXA,REDE OPTICA,CONCLUIDA,,,CLARO BRASIL,DM4001_CANCL0-01,MAPINFO,9/2/2024 101329 PM,,,,,,,RSCAN12,,,,,,,,,,,
566,7453939,7399553,CAN C05,RS,SPS,ANEL_CANMBO-03,CAN MBO,GPON MOVEL,REDE OPTICA,AVALIAR CAPACIDADE,,GPON,CLARO BRASIL,DM4001_CANMBO-01,MAPINFO,9/2/2024 101329 PM,,,,CAPACIDADE_BB,,22450-27946,,,,,,,,,,,,PARKS20048S-CANC05-001


In [24]:
resumosoe[resumosoe.SEV == 7453939].FO_EDD_ETH_INFORMACAO.values

array(['NUVEM:  / SLEJC_ANEL-01 / ANEL_CANMBO / ANEL_CANMBO-04'],
      dtype=object)